# ElecForecast - Prediction 

In [2]:
# !pip install -Uqq fastai
# !pip install -Uqq xgboost

import requests
import json
import pandas as pd
import numpy as np
import math
import io
import shutil
import zipfile 
import os
import sys
from pathlib import Path
import pickle
from datetime import datetime, timedelta, date
import timeit
np.random.seed(2)
import matplotlib.pyplot as plt
from scipy.stats import logistic

# from sklearn import preprocessing
# from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error
import joblib
import xgboost
# from fastai.tabular.all import *
import fastai.tabular.all as fastai

from aemo import *
    
# Display floats in this notbook to 3 decimal places
pd.set_option('display.float_format', lambda x: '%.3f' % x)
plt.rcParams["figure.figsize"] = (16,10)

# Constants
MODELS_FOLDER = Path('../models')
REGIONIDS = ['NSW1', 'QLD1', 'SA1', 'TAS1', 'VIC1']
FORECAST_TIMES = list(range(2,24,2)) + list(range(24,168+1,4))  # every 2hrs for 24hrs, then every 4hrs to 168 (week)
ENSEMBLE_RATIO = 0.6  # nn * ENSEMBLE_RATIO  +  xgb (1-ENSEMBLE_RATIO) == prediction
p_min, p_max = -400, 1000 # values to clip price columns to when decoding
# list of forecasts to make, eg 'VIC1_Price' where forecast is a list of predictions at FORECAST_TIMES (ie up to a week out)
FORECASTS_TO_MAKE = [f"{region}_{price_or_greenness}" for region in REGIONIDS for price_or_greenness in ['Price', 'Greenness']]


In [3]:
duids = get_duids()

Already downloaded ../data/download_cache/duids_from_neo.json


### NEM Data Helper Functions

In [ ]:


# Global Constants
REGIONIDS = ['NSW1', 'QLD1', 'SA1', 'TAS1', 'VIC1']
INTERCONNECTORIDS = ['N-Q-MNSP1', 'NSW1-QLD1', 'T-V-MNSP1', 'V-S-MNSP1', 'V-SA', 'VIC1-NSW1']
MAX_CSV_FILE_SIZE_TO_KEEP_ON_DISK_IN_BYTES = 10000000 
DATA_FOLDER = Path('../data')
DOWNLOAD_CACHE_FOLDER = DATA_FOLDER / 'download_cache'


def ppjson(x):
    if isinstance(x, str):
        print(json.dumps(json.loads(x), indent=2))
    if isinstance(x, dict) or isinstance(x, list):
        print(json.dumps(x, indent=2))

# often need to turn 5-minute data into 1-hour data. 
# All timestamps in dataset are at the END of a period, so can't just .replace(minute=0).
# We want to round UP to next hour. 
# round_up_to_hour = lambda x: (x+timedelta(minutes=55)).replace(minute=0)
def round_up_to_hour(df, column):
    df[column] = df[column].apply(lambda x: (x+timedelta(minutes=55)).replace(minute=0))
    return df

# print ALL columns from just two rows, displays all even when pandas cuts everything off.
def print_item(df, index=0, offset=1):
    cells = list(zip([x for x in df.columns],[x for x in df.iloc[index]],[x for x in df.iloc[index+offset]]))
    [print(f'{a[0]:30}{a[1]:<30}{a[2]:<30}') for a in cells]

def check_nas(df):
    return df.loc[df.isnull().any(axis=1)]


# Get Features

In [ ]:
features = {}

In [ ]:
with open('../data/columns_for_VIC1.json') as f:
    dataset_cols = json.loads(f.read())
# dataset_cols

In [ ]:
set(dataset_cols) - set(features.keys())

In [ ]:
columns = [
    # Day features - calculatd
    'day', 'hour', 'hours_since_2010', 'is_weekend', 'month', 'quarter', 'weekday', 'year', 

    # Holidays - from scraped holidays data
    'RG_is_holiday', 
    'RG_is_workday',   'RG_is_workday_Tp24', 'RG_is_workday_Tp48', 'RG_is_workday_Tp72', 'RG_is_workday_Tp96', 'RG_is_workday_Tp120', 'RG_is_workday_Tp144', 'RG_is_workday_Tp168', 

# Weather
    'RG_W_temperature', 
    'RG_W_day_max_temperature', 'RG_W_day_max_temperature_Tp24', 'RG_W_day_max_temperature_Tp48', 'RG_W_day_max_temperature_Tp72', 'RG_W_day_max_temperature_Tp96', 'RG_W_day_max_temperature_Tp120', 'RG_W_day_max_temperature_Tp144', 'RG_W_day_max_temperature_Tp168', 
# 'RG_W_sun', 'RG_W_cloud', 'RG_W_wind', 
# 'RG_W_day_max_sun',         'RG_W_day_max_sun_Tp24', 'RG_W_day_max_sun_Tp48', 'RG_W_day_max_sun_Tp72', 'RG_W_day_max_sun_Tp96', 'RG_W_day_max_sun_Tp120', 'RG_W_day_max_sun_Tp144', 'RG_W_day_max_sun_Tp168', 
# 'RG_W_day_max_wind',        'RG_W_day_max_wind_Tp24','RG_W_day_max_wind_Tp48', 'RG_W_day_max_wind_Tp72', 'RG_W_day_max_wind_Tp96', 'RG_W_day_max_wind_Tp120', 'RG_W_day_max_wind_Tp144', 'RG_W_day_max_wind_Tp168', 

    # Price & generation by region - From various sub-tables in "DISPATCHIS" 
    'RG_Price', 
    'RG_GENERATION', 'RG_AVAILABLEGENERATION', 'RG_TOTALDEMAND',
    'RG_IC_NET', 'RG_IC_Export_Limit', 'RG_IC_Import_Limit', 
    
    # Price lags - using table "TradingIS_Reports"
    'RG_Price_Tm1', 'RG_Price_Tm2', 'RG_Price_Tm3', 'RG_Price_Tm4', 'RG_Price_Tm6', 'RG_Price_Tm8', 'RG_Price_Tm12', 'RG_Price_Tm16', 'RG_Price_Tm20', 'RG_Price_Tm24', 
    'RG_Price_Tm36', 'RG_Price_Tm48', 'RG_Price_Tm168', 
    'RG_Price_Tm30d_25thP', 'RG_Price_Tm30d_Median', 'RG_Price_Tm30d_75thP', 'RG_Price_Tm30d_Mean', 

    # Forecasts of Region stats:
    # PREDISPATCHIS table > REGION_SOLUTION subtable until 4.30 day-after-next trading day, jioned with STPASA table, REGIONSOLUTION sub-table for the remainder of 7 days ahead
    'RG_AVAILABLEGENERATION_Tp6', 'RG_AVAILABLEGENERATION_Tp12', 'RG_AVAILABLEGENERATION_Tp18', 'RG_AVAILABLEGENERATION_Tp24', 'RG_AVAILABLEGENERATION_Tp30', 'RG_AVAILABLEGENERATION_Tp36', 'RG_AVAILABLEGENERATION_Tp42', 'RG_AVAILABLEGENERATION_Tp48', 'RG_AVAILABLEGENERATION_Tp54', 'RG_AVAILABLEGENERATION_Tp60', 'RG_AVAILABLEGENERATION_Tp66', 'RG_AVAILABLEGENERATION_Tp72', 'RG_AVAILABLEGENERATION_Tp78', 'RG_AVAILABLEGENERATION_Tp84', 'RG_AVAILABLEGENERATION_Tp90', 'RG_AVAILABLEGENERATION_Tp96', 'RG_AVAILABLEGENERATION_Tp102', 'RG_AVAILABLEGENERATION_Tp108', 'RG_AVAILABLEGENERATION_Tp114', 'RG_AVAILABLEGENERATION_Tp120', 'RG_AVAILABLEGENERATION_Tp126', 'RG_AVAILABLEGENERATION_Tp132', 'RG_AVAILABLEGENERATION_Tp138', 'RG_AVAILABLEGENERATION_Tp144', 'RG_AVAILABLEGENERATION_Tp150', 'RG_AVAILABLEGENERATION_Tp156', 'RG_AVAILABLEGENERATION_Tp162', 'RG_AVAILABLEGENERATION_Tp168', 
    'RG_TOTALDEMAND_Tp6', 'RG_TOTALDEMAND_Tp12', 'RG_TOTALDEMAND_Tp18', 'RG_TOTALDEMAND_Tp24', 'RG_TOTALDEMAND_Tp30', 'RG_TOTALDEMAND_Tp36', 'RG_TOTALDEMAND_Tp42', 'RG_TOTALDEMAND_Tp48', 'RG_TOTALDEMAND_Tp54', 'RG_TOTALDEMAND_Tp60', 'RG_TOTALDEMAND_Tp66', 'RG_TOTALDEMAND_Tp72', 'RG_TOTALDEMAND_Tp78', 'RG_TOTALDEMAND_Tp84', 'RG_TOTALDEMAND_Tp90', 'RG_TOTALDEMAND_Tp96', 'RG_TOTALDEMAND_Tp102', 'RG_TOTALDEMAND_Tp108', 'RG_TOTALDEMAND_Tp114', 'RG_TOTALDEMAND_Tp120', 'RG_TOTALDEMAND_Tp126', 'RG_TOTALDEMAND_Tp132', 'RG_TOTALDEMAND_Tp138', 'RG_TOTALDEMAND_Tp144', 'RG_TOTALDEMAND_Tp150', 'RG_TOTALDEMAND_Tp156', 'RG_TOTALDEMAND_Tp162', 'RG_TOTALDEMAND_Tp168', 
    'RG_GEN_Solar_Tp6', 'RG_GEN_Solar_Tp12', 'RG_GEN_Solar_Tp18', 'RG_GEN_Solar_Tp24', 'RG_GEN_Solar_Tp30', 'RG_GEN_Solar_Tp36', 'RG_GEN_Solar_Tp42', 'RG_GEN_Solar_Tp48', 'RG_GEN_Solar_Tp54', 'RG_GEN_Solar_Tp60', 'RG_GEN_Solar_Tp66', 'RG_GEN_Solar_Tp72', 'RG_GEN_Solar_Tp78', 'RG_GEN_Solar_Tp84', 'RG_GEN_Solar_Tp90', 'RG_GEN_Solar_Tp96', 'RG_GEN_Solar_Tp102', 'RG_GEN_Solar_Tp108', 'RG_GEN_Solar_Tp114', 'RG_GEN_Solar_Tp120', 'RG_GEN_Solar_Tp126', 'RG_GEN_Solar_Tp132', 'RG_GEN_Solar_Tp138', 'RG_GEN_Solar_Tp144', 'RG_GEN_Solar_Tp150', 'RG_GEN_Solar_Tp156', 'RG_GEN_Solar_Tp162', 'RG_GEN_Solar_Tp168', 
    'RG_GEN_Wind_Tp6', 'RG_GEN_Wind_Tp12', 'RG_GEN_Wind_Tp18', 'RG_GEN_Wind_Tp24', 'RG_GEN_Wind_Tp30', 'RG_GEN_Wind_Tp36', 'RG_GEN_Wind_Tp42', 'RG_GEN_Wind_Tp48', 'RG_GEN_Wind_Tp54', 'RG_GEN_Wind_Tp60', 'RG_GEN_Wind_Tp66', 'RG_GEN_Wind_Tp72', 'RG_GEN_Wind_Tp78', 'RG_GEN_Wind_Tp84', 'RG_GEN_Wind_Tp90', 'RG_GEN_Wind_Tp96', 'RG_GEN_Wind_Tp102', 'RG_GEN_Wind_Tp108', 'RG_GEN_Wind_Tp114', 'RG_GEN_Wind_Tp120', 'RG_GEN_Wind_Tp126', 'RG_GEN_Wind_Tp132', 'RG_GEN_Wind_Tp138', 'RG_GEN_Wind_Tp144', 'RG_GEN_Wind_Tp150', 'RG_GEN_Wind_Tp156', 'RG_GEN_Wind_Tp162', 'RG_GEN_Wind_Tp168', 

    # Availability by fuel type - use 24h ago (because now isn't available). 
    'RG_AVAILABILITY_Battery Storage', 'RG_AVAILABILITY_Coal', 'RG_AVAILABILITY_Gas', 'RG_AVAILABILITY_Hydro', 'RG_AVAILABILITY_Solar', 'RG_AVAILABILITY_Wind', 

    # From 'DISPATCHSCADA': generation by fuel type
    'RG_GEN_Battery Storage', 'RG_GEN_Coal', 'RG_GEN_Gas', 'RG_GEN_Hydro', 'RG_GEN_Solar', 'RG_GEN_Wind', 

    # ROOFTOP_PV_ACTUAL / ROOFTOP_PV_FORECAST
    'RG_GEN_Rooftop', 
    'RG_GEN_Rooftop_Tp6', 'RG_GEN_Rooftop_Tp12', 'RG_GEN_Rooftop_Tp18', 'RG_GEN_Rooftop_Tp24', 'RG_GEN_Rooftop_Tp30', 'RG_GEN_Rooftop_Tp36', 'RG_GEN_Rooftop_Tp42', 'RG_GEN_Rooftop_Tp48', 'RG_GEN_Rooftop_Tp54', 'RG_GEN_Rooftop_Tp60', 'RG_GEN_Rooftop_Tp66', 'RG_GEN_Rooftop_Tp72', 'RG_GEN_Rooftop_Tp78', 'RG_GEN_Rooftop_Tp84', 'RG_GEN_Rooftop_Tp90', 'RG_GEN_Rooftop_Tp96', 'RG_GEN_Rooftop_Tp102', 'RG_GEN_Rooftop_Tp108', 'RG_GEN_Rooftop_Tp114', 'RG_GEN_Rooftop_Tp120', 'RG_GEN_Rooftop_Tp126', 'RG_GEN_Rooftop_Tp132', 'RG_GEN_Rooftop_Tp138', 'RG_GEN_Rooftop_Tp144', 'RG_GEN_Rooftop_Tp150', 'RG_GEN_Rooftop_Tp156', 'RG_GEN_Rooftop_Tp162', 'RG_GEN_Rooftop_Tp168', 

    # TODO: ... but IC_NET is hard... not sure I can trust that data. Maybe change the model, should have used import/export limits not IC_NET. 
# 'RG_IC_NET_Tp6', 'RG_IC_NET_Tp12', 'RG_IC_NET_Tp18', 'RG_IC_NET_Tp24', 'RG_IC_NET_Tp30', 'RG_IC_NET_Tp36', 'RG_IC_NET_Tp42', 'RG_IC_NET_Tp48', 'RG_IC_NET_Tp54', 'RG_IC_NET_Tp60', 'RG_IC_NET_Tp66', 'RG_IC_NET_Tp72', 'RG_IC_NET_Tp78', 'RG_IC_NET_Tp84', 'RG_IC_NET_Tp90', 'RG_IC_NET_Tp96', 'RG_IC_NET_Tp102', 'RG_IC_NET_Tp108', 'RG_IC_NET_Tp114', 'RG_IC_NET_Tp120', 'RG_IC_NET_Tp126', 'RG_IC_NET_Tp132', 'RG_IC_NET_Tp138', 'RG_IC_NET_Tp144', 'RG_IC_NET_Tp150', 'RG_IC_NET_Tp156', 'RG_IC_NET_Tp162', 'RG_IC_NET_Tp168'
    
    # Calculated:
    'RG_Greenness', 
    'RG_Greenness_Tm30d_Mean', 

]

## Date Features
'day', 'hour', 'hours_since_2010', 'is_weekend', 'month', 'quarter', 'weekday', 'year',

In [ ]:
def get_date_features():
    output = {}
    now = pd.Timestamp.now(tz='Australia/Brisbane').to_pydatetime()
    output['year'] = now.year - 2010
    output['month'] = now.month
    output['day'] = now.day
    output['hour'] = now.hour
    output['weekday'] = now.weekday()
    output['quarter'] = ((now.month-1) // 3)+1 + (now.year-2010) * 4
    output['is_weekend'] = now.weekday() >= 5


    # hours_since_2010: note, not actually hours, it's timesteps, and they are now 5 min timesteps. 
    # also it's rounded to 3 sig figs as a float... wonder if that's enough...
    # 9 Jan 2010 happens to equal 2020
    START_DATE = datetime(2010, 1, 2)

    def datetime_to_x_since_2010(dt): 
        delta = dt.replace(tzinfo=None) - START_DATE
        x = delta.total_seconds() / 60 / 5
        return int(float('%.3g' % x))
    # tests from dataset6
    assert datetime_to_x_since_2010(datetime(2010, 1, 9)) == 2020
    assert datetime_to_x_since_2010(datetime(2022, 6, 20, 10)) == 1310000
    assert datetime_to_x_since_2010(datetime(2010, 1, 15, 22, 34, 0)) == 4010
    assert datetime_to_x_since_2010(datetime(2010, 1, 15, 22, 35, 0)) == 4020

    output['hours_since_2010'] = datetime_to_x_since_2010(now)
    
    return output

features = features | output

## Holidays, workdays, workday forecasts

In [ ]:

def get_workday_features()
    """Get features for holidays and workdays"""
    output = {}
    holidays = pd.read_csv(DATA_FOLDER / 'australian_public_holidays_scraped_2010-2024.csv', parse_dates=['Date'], index_col=0)
    holidays = ~holidays.pivot(index='Date', columns='REGIONID').isna()
    holidays.columns = [f"{x[1]}_is_holiday" for x in holidays.columns]

    def get_is_holiday(date, region):
        date_str = date.isoformat()
        if date_str in holidays.index:
            region_is_holiday = holidays.loc[date_str, f'{region}_is_holiday']
        else:
            region_is_holiday = False
        return region_is_holiday

    assert get_is_holiday(date(2010, 1, 26), 'SA1') == True
    assert get_is_holiday(date(2010, 1, 30), 'QLD1') == False
    assert get_is_holiday(date(2024, 11, 4), 'NSW1') == False
    assert get_is_holiday(date(2024, 11, 5), 'VIC1') == True


    # NEM time = AEST = Brisbane, because they don't have daylight savings time
    today = pd.Timestamp.now(tz='Australia/Brisbane').normalize().to_pydatetime().date()

    for region in REGIONIDS:
        output[f'{region}_is_holiday'] = get_is_holiday(today, region)
        output[f'{region}_is_workday'] = not (output['is_weekend'] or output[f'{region}_is_holiday'])

    next_7_days = [today + timedelta(days=x) for x in range(1, 7+1)]

    is_weekend_forecast = [day.weekday() >= 5 for day in next_7_days]

    for region in REGIONIDS:
        is_holiday_forecast = [get_is_holiday(day, region) for day in next_7_days]
        is_workday_forecast = [not(weekend or holiday) for weekend, holiday in list(zip(is_weekend_forecast, is_holiday_forecast))]
        is_workday_forecast

        for i, is_workday in enumerate(is_workday_forecast):
            output[f'{region}_is_workday_Tp{24 * (i+1)}'] = is_workday

## Weather Forecasts
- 'RG_W_temperature', 'RG_W_wind', 
- 'RG_W_sun', 'RG_W_cloud', 
- 'RG_W_day_max_temperature', 'RG_W_day_max_temperature_Tp24', 'RG_W_day_max_temperature_Tp48',  'RG_W_day_max_temperature_Tp72', 'RG_W_day_max_temperature_Tp96', 'RG_W_day_max_temperature_Tp120',  'RG_W_day_max_temperature_Tp144', 'RG_W_day_max_temperature_Tp168', 


In [ ]:
def get_weather_features():
    """get max temp forecasts from the BoM
    
    Data comes from the api used in https://weather.bom.gov.au/location/r1r0fsn-melbourne and similar pages
    Dodgy: sometimes is short one day. See below
    """
    bom_codes = {
        'NSW1': 'r3gx2f',
        'VIC1': 'r1r0fs',
        'TAS1': 'r22u09',
        'QLD1': 'r7hgdp',
        'SA1': 'r1f93c',
    }
    output = {}
    for region in REGIONIDS:
    # for region in ['SA1']:

        # get current temperature
        r = requests.get(f'https://api.weather.bom.gov.au/v1/locations/{bom_codes[region]}/observations')
        observation = r.json()
        output[f'{region}_W_temperature'] = observation['data']['temp']
        max_temp_today = observation['data']['max_temp']['value']
    
        # get forecast temperatures
        r = requests.get(f'https://api.weather.bom.gov.au/v1/locations/{bom_codes[region]}/forecasts/daily')
        forecast = r.json()

        # extract date & max_temp from the BoM data.  Round to nearest day to avoid having to deal with DST or each city's particular timezone.
        temps_lookup = {pd.Timestamp(x['date']).tz_convert('Australia/Brisbane').round('D').tz_localize(tz=None): x['temp_max'] 
                        for x in forecast['data']}

        # need to match those dates against the desired dates, which start from 'today' (regardless of current time) and then 7 more days after that.
        today = (pd.Timestamp.utcnow()
                 .tz_convert('Australia/Brisbane')  # Use Brisbane tz beacuse always AEST (no DST)
                 .floor('D')
                 .tz_localize(tz=None))  # remove timezone
        days = [today + pd.Timedelta(x, 'D') for x in range(8)]
        if today not in temps_lookup: temps_lookup[today] = None
        if days[-1] not in temps_lookup: temps_lookup[days[-1]] = None
        temps = [temps_lookup[day] for day in days]
        # dodgy hack if there's one day short: copy from previous day
        # dataset7 has current day's max temp (depending on current time, this could be future or 
        # past) and requests the next 7 days ... but early in the morning, there aren't this many days forecast by bom, it returns 'None' in last day.
        # So just copy the prev day's prediction. 
        if temps[-1] is None: temps[-1] = temps[-2] 
        #  if today is missing (not sure if that actually happens) then replace it with observed data from today
        if temps[0] is None: temps[0] = max_temp_today
        # temps[0] = max(temps[0], max_temp_today)  # don't do this because observations may be for yesterday (they are at 1am)
        
        assert None not in temps, f"something has gone wrong with weather forecast data for {region}. Max temps are {temps}"
        assert len(temps) == 8, f"there should be 8 forecast {region} temperatures but there are {len(temps)}"

        temp_names = ['RG_W_day_max_temperature', 'RG_W_day_max_temperature_Tp24', 'RG_W_day_max_temperature_Tp48', 'RG_W_day_max_temperature_Tp72', 'RG_W_day_max_temperature_Tp96', 'RG_W_day_max_temperature_Tp120', 'RG_W_day_max_temperature_Tp144', 'RG_W_day_max_temperature_Tp168']
        temp_names = [x.replace('RG', region) for x in temp_names]
        
        for name, temp in list(zip(temp_names, temps)):
            output[name] = temp

    return output

features = features | get_weather_features()
features
        

## Price & Generation by region

Various sub-tables of "Dispatch", which are equivalent of `DISPATCHREGIONSUM`, `DISPATCHPRICE` and `DISPATCHINTERCONNECTORRES` in the dataset generator.

In [ ]:
def get_price_and_gen():
    """Get Price & Generation features by region

    Downloads various sub-tables of "Dispatch", which are equivalent of `DISPATCHREGIONSUM`, 
    `DISPATCHPRICE` and `DISPATCHINTERCONNECTORRES` in the dataset generator.
    """
    filename = get_latest_file_from_folder('DispatchIS_Reports')

    # price features
    price_table = pd.read_csv(get_subtable_from_file(filename, 'DISPATCH,PRICE,'), 
                              index_col='REGIONID', usecols=['REGIONID', 'RRP'])
    for region in REGIONIDS:
        features[f'{region}_Price'] = price_table.at[region, 'RRP']

    # regionsum features aka 'DISPATCHREGIONSUM' aka 'Generation For Each Region'
    regionsum_table = pd.read_csv(get_subtable_from_file(filename, 'DISPATCH,REGIONSUM,'),
                                  index_col='REGIONID', 
                                  usecols=['REGIONID', 'AVAILABLEGENERATION', 'TOTALDEMAND', 'DISPATCHABLEGENERATION', 'NETINTERCHANGE'])
    # IC_NET is defined to be NETINTERCHANGE * -1. IC_NET: import is +ve generation.
    regionsum_table['NETINTERCHANGE'] = regionsum_table['NETINTERCHANGE'] * -1
    regionsum_table = regionsum_table.rename(columns={'NETINTERCHANGE': 'IC_NET', 'DISPATCHABLEGENERATION': 'GENERATION' })
    for region in REGIONIDS:
        for col in regionsum_table.columns:
            features[f'{region}_{col}'] = regionsum_table.at[region, col]

    # Interconnector import / export limits
    ic_table = pd.read_csv(get_subtable_from_file(filename, 'DISPATCH,INTERCONNECTORRES,'), 
                           usecols=['INTERCONNECTORID', 'EXPORTLIMIT', 'IMPORTLIMIT'], 
                           index_col='INTERCONNECTORID')
    # manually convert each interconnector data to region summaries
    features['VIC1_IC_Import_Limit'] = (- ic_table.at['T-V-MNSP1', 'EXPORTLIMIT'] + ic_table.at['V-S-MNSP1', 'IMPORTLIMIT'] + ic_table.at['V-SA', 'IMPORTLIMIT'] + ic_table.at['VIC1-NSW1', 'IMPORTLIMIT']) * -1
    features['VIC1_IC_Export_Limit'] = (- ic_table.at['T-V-MNSP1', 'IMPORTLIMIT'] + ic_table.at['V-S-MNSP1', 'EXPORTLIMIT'] + ic_table.at['V-SA', 'EXPORTLIMIT'] + ic_table.at['VIC1-NSW1', 'EXPORTLIMIT']) * -1
    features['TAS1_IC_Import_Limit'] = ic_table.at['T-V-MNSP1', 'IMPORTLIMIT'] * -1
    features['TAS1_IC_Export_Limit'] = ic_table.at['T-V-MNSP1', 'EXPORTLIMIT'] * -1
    features['SA1_IC_Import_Limit']  = (- ic_table.at['V-S-MNSP1', 'EXPORTLIMIT'] - ic_table.at['V-SA', 'EXPORTLIMIT']) * -1
    features['SA1_IC_Export_Limit']  = (- ic_table.at['V-S-MNSP1', 'IMPORTLIMIT'] - ic_table.at['V-SA', 'IMPORTLIMIT']) * -1
    features['NSW1_IC_Import_Limit'] = (- ic_table.at['VIC1-NSW1', 'EXPORTLIMIT'] + ic_table.at['NSW1-QLD1', 'IMPORTLIMIT'] + ic_table.at['N-Q-MNSP1', 'IMPORTLIMIT']) * -1
    features['NSW1_IC_Export_Limit'] = (- ic_table.at['VIC1-NSW1', 'IMPORTLIMIT'] + ic_table.at['NSW1-QLD1', 'EXPORTLIMIT'] + ic_table.at['N-Q-MNSP1', 'EXPORTLIMIT']) * -1
    features['QLD1_IC_Import_Limit'] = (- ic_table.at['NSW1-QLD1', 'EXPORTLIMIT'] - ic_table.at['N-Q-MNSP1', 'EXPORTLIMIT']) * -1
    features['QLD1_IC_Export_Limit'] = (- ic_table.at['NSW1-QLD1', 'IMPORTLIMIT'] - ic_table.at['N-Q-MNSP1', 'IMPORTLIMIT']) * -1
    return features
features = features | get_price_and_gen()
features

## Price Lags - up to 30d in the past

The last 24 hours of lags aren't necessarily in the daily reports, so get them from 5-minute reports. 

Note: Not bothering to do any averaging across the whole hour, just instantaneous 5-min values. 

In [ ]:
def get_price_lags():
    """Get all the price lag features."""
    output = {}

    # The last 24 hours of lags aren't necessarily in the daily reports, so get them from 5-minute reports.
    # Note: Not bothering to do any averaging across the whole hour, just instantaneous 5-min values.
    lags = [1, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
    targets = [f'RG_Price_Tm{hours}' for hours in lags]
    for lag in lags:
        filename = get_latest_file_from_folder('TradingIS_Reports', nth_most_recent=-1*lag*12)
        df = pd.read_csv(get_subtable_from_file(filename, ',TRADING,PRICE,'),
                         usecols=['REGIONID', 'RRP', ],
                         index_col='REGIONID'
                        )
        for region in REGIONIDS:
            output[f"{region}_Price_Tm{lag}"] = df.at[region, 'RRP']
            
    # Get the remaining lags up to 30 days ago from the daily reports
    # 'RG_Price_Tm28', 'RG_Price_Tm32',... 'RG_Price_Tm168',
    # 'RG_Price_Tm30d_25thP', 'RG_Price_Tm30d_Median', 'RG_Price_Tm30d_75thP', 'RG_Price_Tm30d_Mean'
    # Firest, get last 30 days of daily reports:
    filenames =[get_latest_file_from_folder('Daily_Reports', nth_most_recent=i) 
                for i in range(-1, -31, -1)]

    # Get prices, averaged to hourly, for the last 30 days
    dfs = []
    for filename in filenames:
        df = pd.read_csv(get_subtable_from_file(filename, ',DREGION,,3'),
                         usecols=['REGIONID', 'RRP', 'SETTLEMENTDATE', ],
                         parse_dates=['SETTLEMENTDATE'],
                         index_col='SETTLEMENTDATE'
                        )
        df = df.pivot(columns='REGIONID', values='RRP')
        # change from 5min to 60min frequency to match PV_forecast
        df = df.resample('H', origin='start').mean()
        df.index = df.index + pd.tseries.frequencies.to_offset('55min')  # because SETTLEMENTDATE is end-of-period

        dfs.append(df)

    # concat all
    df = pd.concat(dfs)

    ### outputs:

    # 'RG_Price_Tm28', 'RG_Price_Tm32',... 'RG_Price_Tm168',
    for lag in [28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 168]:
        for region in REGIONIDS:
            output[f"{region}_Price_Tm{lag}"] = df.at[pd.Timestamp.now().round('H') - pd.Timedelta(lag, 'H'), region]
    
    # 'RG_Price_Tm7d_25thP', 'RG_Price_Tm7d_Median', 'RG_Price_Tm7d_75thP'
    for region in REGIONIDS:
        last_7_days = df[region][df.index > pd.Timestamp.now() - pd.Timedelta(7, 'D')]
        output[f'{region}_Price_Tm7d_15thP'] = last_7_days.quantile(0.15)
        output[f'{region}_Price_Tm7d_Median'] = last_7_days.quantile(0.5)
        output[f'{region}_Price_Tm7d_85thP'] = last_7_days.quantile(0.85)

    # 'RG_Price_Tm30d_25thP', 'RG_Price_Tm30d_Median', 'RG_Price_Tm30d_75thP'
    for region in REGIONIDS:
        output[f'{region}_Price_Tm30d_15thP'] = df[region].quantile(0.15)
        output[f'{region}_Price_Tm30d_Median'] = df[region].quantile(0.5)
        output[f'{region}_Price_Tm30d_85thP'] = df[region].quantile(0.85)
    
    return output

features = features | get_price_lags()

## Pre-dispatch Prices

In [ ]:
def get_predispatch_prices():
    """Get predispatch price prediction features
    
    'VIC1_Predis_Price_Tp4', 'VIC1_Predis_Price_Tp8', 'VIC1_Predis_Price_Tp12',  'VIC1_Predis_Price_Tp16',
    'VIC1_Predis_Price_max16to40h', 'VIC1_Predis_Price_min16to40h'
    """
    filename = get_latest_file_from_folder('PredispatchIS_Reports')
    df = pd.read_csv(get_subtable_from_file(filename, 'PREDISPATCH,REGION_PRICES'),
                     usecols=['REGIONID', 'RRP', 'DATETIME'],
                     parse_dates=['DATETIME'],)
                     # index_col='DATETIME')

    # removes final rows with NAs
    print(df.shape)
    # Duplicated rows ... probably because of multiple (types of?) runs for same period. Keep only the last ones. 
    df = df.drop_duplicates(['DATETIME', 'REGIONID'], keep='last')
    # Don't need half-hourly, hourly is fine, drop the other rows
    df = df[df['DATETIME'].dt.minute == 0]

    # do the pivot
    now = df.iloc[0].DATETIME
    df['Forecast_Distance'] = df['DATETIME'] - now
    df = df.pivot(index='Forecast_Distance', 
                  columns='REGIONID',
                  values='RRP'
                 ).sort_index(axis=1, level='REGIONID')
    df.index = df.index.total_seconds() // 3600

    output = {}
    for region in REGIONIDS:
        for lag in [4, 8, 12, 16]:
            output[f'{region}_Predis_Price_Tp{lag}'] = df[region].loc[lag]
        output[f'{region}_Predis_Price_max16to40h'] = df[region].loc[16:44].max()
        output[f'{region}_Predis_Price_min16to40h'] = df[region].loc[16:44].min()
    return output
features = features | get_predispatch_prices()

## Forecasts of Price & Gen by Region

Requires a combination of pre-dispatch (for the next day-or-so) and STPASA (which starts at 4.30am on day after next trading day)
- 'RG_AVAILABLEGENERATION_Tp6', 'RG_AVAILABLEGENERATION_Tp12'...'RG_AVAILABLEGENERATION_Tp168', 
- 'RG_TOTALDEMAND_Tp6', 'RG_TOTALDEMAND_Tp12'... 'RG_TOTALDEMAND_Tp168', 
- 'RG_GEN_Solar_Tp6', 'RG_GEN_Solar_Tp12', ... 'RG_GEN_Solar_Tp168', 
- 'RG_GEN_Wind_Tp6', 'RG_GEN_Wind_Tp12', ... 'RG_GEN_Wind_Tp168', 
  

In [6]:
predis

,NSW1_AVAILABLEGENERATION,QLD1_AVAILABLEGENERATION,SA1_AVAILABLEGENERATION,TAS1_AVAILABLEGENERATION,VIC1_AVAILABLEGENERATION,NSW1_TOTALDEMAND,QLD1_TOTALDEMAND,SA1_TOTALDEMAND,TAS1_TOTALDEMAND,VIC1_TOTALDEMAND,NSW1_GEN_Solar,QLD1_GEN_Solar,SA1_GEN_Solar,TAS1_GEN_Solar,VIC1_GEN_Solar,NSW1_GEN_Wind,QLD1_GEN_Wind,SA1_GEN_Wind,TAS1_GEN_Wind,VIC1_GEN_Wind
DATETIME,,,,,,,,,,,,,,,,,,,,
2022-10-05 12:30:00,10765.050,8946.387,1934.369,2240.866,9076.913,8130.350,5009.350,1166.470,1104.000,5456.200,496.022,1321.744,220.511,0.000,137.262,1211.028,154.643,296.858,211.866,2116.651
2022-10-05 13:00:00,10717.369,8906.408,2014.529,2235.231,8774.439,8102.040,5065.500,1133.430,1077.000,5401.020,447.007,1291.293,227.712,0.000,126.049,1212.362,142.115,368.817,206.231,2107.390
2022-10-05 13:30:00,10715.147,8972.115,2063.701,2205.494,8940.591,8119.820,5215.050,1063.250,1046.000,5379.560,439.765,1208.926,237.183,0.000,128.198,1216.382,145.189,410.518,204.494,2115.393
2022-10-05 14:00:00,10701.405,8902.548,2091.244,2203.250,8956.163,8158.600,5373.160,1028.240,1035.000,5372.330,417.931,1081.243,235.381,0.000,130.048,1223.474,151.305,442.863,202.250,2129.115
2022-10-05 14:30:00,10866.202,8779.424,2121.812,2226.560,8969.685,8195.880,5535.740,1005.640,1028.000,5365.550,384.167,941.403,225.720,0.000,124.838,1226.035,163.021,468.092,203.560,2148.847
2022-10-05 15:00:00,10821.944,8987.615,2148.397,2228.309,9276.916,8281.450,5745.930,1002.610,1032.000,5411.890,337.472,805.239,230.460,0.000,116.731,1228.472,177.376,490.937,205.309,2187.185
2022-10-05 15:30:00,10780.983,8782.219,2353.836,2348.513,9406.074,8403.700,5950.120,1031.530,1044.000,5496.090,291.108,565.523,264.814,0.000,105.547,1241.875,209.696,511.022,207.513,2267.527
2022-10-05 16:00:00,10942.025,9106.621,2426.478,2367.113,9445.230,8559.040,6204.390,1069.000,1073.000,5587.400,239.314,450.004,260.899,0.000,91.830,1224.711,224.617,532.579,211.113,2313.400
2022-10-05 16:30:00,11032.221,9433.925,2433.195,2431.000,9466.678,8704.340,6436.840,1140.100,1103.000,5715.050,171.940,319.109,251.468,0.000,75.270,1238.281,232.816,487.727,217.000,2334.408


In [20]:
def get_price_gen_forecasts():
    """Get features for forecasts of Price & Gen by Region.

    Combination pre-dispatch (for the next day-or-so) and STPASA (which starts at 4.30am on day after next trading day)
    Outputs dict of features:
        'RG_AVAILABLEGENERATION_Tp6', 'RG_AVAILABLEGENERATION_Tp12'...'RG_AVAILABLEGENERATION_Tp168',
        'RG_TOTALDEMAND_Tp6', 'RG_TOTALDEMAND_Tp12'... 'RG_TOTALDEMAND_Tp168',
        'RG_GEN_Solar_Tp6', 'RG_GEN_Solar_Tp12', ... 'RG_GEN_Solar_Tp168',
        'RG_GEN_Wind_Tp6', 'RG_GEN_Wind_Tp12', ... 'RG_GEN_Wind_Tp168',
    """
    output = {}
    ### First, we get predispatch data, which covers until the end of the next trading day (4am)
    filename = get_latest_file_from_folder('PredispatchIS_Reports')
    predis = pd.read_csv(get_subtable_from_file(filename, 'PREDISPATCH,REGION_SOLUTION'),
                        usecols=['REGIONID', 'DATETIME', 'AVAILABLEGENERATION', 'TOTALDEMAND', 
                                'NETINTERCHANGE', 'SS_SOLAR_UIGF', 'SS_WIND_UIGF'],
                        parse_dates=['DATETIME'],
                        index_col='DATETIME')

    # IC_NET is defined to be NETINTERCHANGE * -1. IC_NET: import is +ve generation... though ignoring IC_net for now
    predis['NETINTERCHANGE'] = predis['NETINTERCHANGE'] * -1
    predis = predis.rename(columns={
        'NETINTERCHANGE': 'IC_NET',
        'SS_SOLAR_UIGF': 'GEN_Solar', 
        'SS_WIND_UIGF': 'GEN_Wind', 
    })
    predis = predis.pivot(columns='REGIONID', values=['AVAILABLEGENERATION', 'TOTALDEMAND', 'GEN_Solar', 'GEN_Wind'])
    predis.columns = [f'{region}_{col}' for col, region in predis.columns]


    ### Second, we get ST PASA data, which covers the remainder of the 7 days
    filename = get_latest_file_from_folder('Short_Term_PASA_Reports')
    stpasa = pd.read_csv(get_subtable_from_file(filename, 'STPASA,REGIONSOLUTION'),
                        usecols=['REGIONID', 'INTERVAL_DATETIME', 'AGGREGATECAPACITYAVAILABLE', 'DEMAND50', 
                                'SS_SOLAR_UIGF', 'SS_WIND_UIGF'],
                        parse_dates=['INTERVAL_DATETIME'],
                    )
    stpasa = stpasa.rename(columns={
        'INTERVAL_DATETIME': 'DATETIME',
        'DEMAND50': 'TOTALDEMAND',
        'AGGREGATECAPACITYAVAILABLE': 'AVAILABLEGENERATION',
        'SS_SOLAR_UIGF': 'GEN_Solar', 
        'SS_WIND_UIGF': 'GEN_Wind', 
    })
    stpasa = stpasa.drop_duplicates(['REGIONID', 'DATETIME'], keep='first')  # there are multiple 'runs' in here, eg "LOR" and "OUTAGE_LRC". "LOR" first
    stpasa = stpasa.set_index('DATETIME')
    stpasa = stpasa.pivot(columns='REGIONID', values=['AVAILABLEGENERATION', 'TOTALDEMAND', 'GEN_Solar', 'GEN_Wind'])
    stpasa.columns = [f'{region}_{col}' for col, region in stpasa.columns]

    gen = pd.concat([predis, stpasa])

    # like done in dataset generator, take the max over each N=6hr increment.
    gen = gen.resample('6H', label='right', closed='right').max()
    print(gen.NSW1_AVAILABLEGENERATION)

    # we don't always have enough data to go right out to 7 days. Worst case is just before the new PASA/predis are released at 1pm NEM time
    # extend the data we do have by copying the final 24h two more times to extend by 48h, which is more than enough (36h would also be enough)
    last_24h = gen[-4:].copy()
    gen = pd.concat([gen, last_24h, last_24h])
    print(gen.NSW1_AVAILABLEGENERATION)


    for i, lag in enumerate(list(range(6, 168+6, 6))):
        for col in gen.columns:
            output[f'{col}_Tp{lag}'] = gen.iloc[i][col]
    return output
features = features | get_price_gen_forecasts()

Already downloaded csv: ../data/download_cache/PUBLIC_PREDISPATCHIS_202210051230_20221005120151.CSV
Already downloaded csv: ../data/download_cache/PUBLIC_STPASA_202210051200_0000000372182536.CSV
DATETIME
2022-10-05 18:00:00   11032.221
2022-10-06 00:00:00   10989.762
2022-10-06 06:00:00    9864.474
2022-10-06 12:00:00    9059.000
2022-10-06 18:00:00    9240.000
2022-10-07 00:00:00    9219.000
2022-10-07 06:00:00    9662.000
2022-10-07 12:00:00   10527.000
2022-10-07 18:00:00   10519.000
2022-10-08 00:00:00   10541.000
2022-10-08 06:00:00   10542.000
2022-10-08 12:00:00   11230.000
2022-10-08 18:00:00   11214.000
2022-10-09 00:00:00   11226.000
2022-10-09 06:00:00   10572.000
2022-10-09 12:00:00   11234.000
2022-10-09 18:00:00   11218.000
2022-10-10 00:00:00   11230.000
2022-10-10 06:00:00   10572.000
2022-10-10 12:00:00   11200.000
2022-10-10 18:00:00   11192.000
2022-10-11 00:00:00   11205.000
2022-10-11 06:00:00   10555.000
2022-10-11 12:00:00   11209.000
2022-10-11 18:00:00   11039.

## Availability by fuel type
- the most up to date availabilty by feul source (up-to-minute data is secret, not available)
- also uses the "daily report", the most recent one, which comes out at 4am each day
- using instantaneous data from 24 hours ago as best estimate for availablity now. Alternative was to use the most recent, which is 4am today. Line ball call, went for this. 

In [ ]:
def get_availability_by_fuel(duids):
    """Get availability features by fuel type.
    
    The most up to date availabilty by feul source (up-to-minute data is secret, not available)
    Also uses the "daily report", the most recent one, which comes out at 4am each day
    Using instantaneous data from 24 hours ago as best estimate for availablity now. Alternative
    was to use the most recent, which is 4am today. Line ball call, went for this. 
    """
    filename = get_latest_file_from_folder('Daily_Reports') 
    df = pd.read_csv(get_subtable_from_file(filename, ',DUNIT,,3'),
                     usecols=['SETTLEMENTDATE', 'DUID', 'AVAILABILITY', ],
                     parse_dates=['SETTLEMENTDATE'],
                     index_col='SETTLEMENTDATE'
                    )
    df = df.join(duids, on='DUID')

    # drop any loads, only want generators
    df = df[df.GENSETTYPE == 'GENERATOR']

    # rows that have same time of day as now (date ignored). 
    df = df[df.index.time == pd.Timestamp.now().round('5min').time()]
    df = df.groupby(['REGIONID', 'CO2E_ENERGY_SOURCE']).agg({
        'AVAILABILITY': np.sum,
    }).reset_index()

    # feature names are eg VIC1_AVAILABILITY_Coal
    df['name'] = df['REGIONID'] + '_AVAILABILITY_' + df['CO2E_ENERGY_SOURCE']
    df = df.set_index('name')

    output = {}
    for region in REGIONIDS:
        for fuel in ['Battery Storage', 'Coal', 'Gas', 'Hydro', 'Solar', 'Wind']:
            col = f'{region}_AVAILABILITY_{fuel}'
            output[col] = df.at[col, 'AVAILABILITY'] if col in df.index else 0
    return output
features = features | get_availability_by_fuel(duids)

## Generation by fuel type
Table `DISPATCHSCADA`, which gets all generation of every plant (DUID). Not that availability isn't provided because it's secret till next day. We don't have access to the table used in dataset generator, `DISPATCHLOAD`. 

In [ ]:
def get_generation_by_fuel(duids):
    """Gets generaetion features broken down by fuel for the current time
    
    Uses table  `DISPATCHSCADA` to get generation for every plant (DUID).
    Note that availability isn't provided because it's secret till next day. 
    We don't have access to the table used in dataset generator, DISPATCHLOAD.
    """
    filename = get_latest_file_from_folder('Dispatch_SCADA')
    df = pd.read_csv(filename, 
                     skiprows=[0, -1], # skip first and last rows
                     usecols=['DUID', 'SCADAVALUE'],
                     index_col='DUID'
                    ).join(duids)

    # drop any loads, only want generators
    df = df[df.GENSETTYPE == 'GENERATOR']

    df = df.groupby(['REGIONID', 'CO2E_ENERGY_SOURCE']).agg({
        'SCADAVALUE': np.sum,
        # 'AVAILABILITY': np.sum,
    }).reset_index()

    # no negatives 
    df['SCADAVALUE'] = df['SCADAVALUE'].clip(lower=0)

    # feature names are eg VIC1_GEN_Coal
    df['name'] = df['REGIONID'] + '_GEN_' + df['CO2E_ENERGY_SOURCE']
    df = df.set_index('name')

    output = {}
    for region in REGIONIDS:
        for fuel in ['Battery Storage', 'Coal', 'Gas', 'Hydro', 'Solar', 'Wind']:
            col = f'{region}_GEN_{fuel}'
            output[col] = df.at[col, 'SCADAVALUE'] if col in df.index else 0
    return output
features = features | get_generation_by_fuel(duids)

## Rooftop PV

In [ ]:
def get_rooftop_pv():
    """Gets features for current and forecast rooftop pv output."""
    # Current values
    filename = get_latest_file_from_folder('ROOFTOP_PV/ACTUAL')
    df = pd.read_csv(filename, 
                     skiprows=1,
                     skipfooter=1,
                     engine='python',
                     usecols=['REGIONID', 'POWER'],
                     index_col='REGIONID')
    current_rooftop = {}
    for region in REGIONIDS:
        current_rooftop[f'{region}_GEN_Rooftop'] = df.at[region, 'POWER']

    # Rooftop PV forecasts
    filename = get_latest_file_from_folder('ROOFTOP_PV/FORECAST')
    df = pd.read_csv(filename, 
                     skiprows=1,
                     skipfooter=1,
                     engine='python',
                     usecols=['INTERVAL_DATETIME', 'REGIONID', 'POWERPOE50'],
                     index_col='INTERVAL_DATETIME')
    df = df.pivot(columns='REGIONID', values='POWERPOE50')

    # like done in dataset generator, take the max over each 6hr increment.
    df = df.rolling(2 * 6).max()
    df = df[11::12]  # take every 12th line, starting from the 11th, which is the end of the first block of 6hrs

    len(['RG_GEN_Rooftop_Tp6', 'RG_GEN_Rooftop_Tp12', 'RG_GEN_Rooftop_Tp18', 'RG_GEN_Rooftop_Tp24', 'RG_GEN_Rooftop_Tp30', 'RG_GEN_Rooftop_Tp36', 'RG_GEN_Rooftop_Tp42', 'RG_GEN_Rooftop_Tp48', 'RG_GEN_Rooftop_Tp54', 'RG_GEN_Rooftop_Tp60', 'RG_GEN_Rooftop_Tp66', 'RG_GEN_Rooftop_Tp72', 'RG_GEN_Rooftop_Tp78', 'RG_GEN_Rooftop_Tp84', 'RG_GEN_Rooftop_Tp90', 'RG_GEN_Rooftop_Tp96', 'RG_GEN_Rooftop_Tp102', 'RG_GEN_Rooftop_Tp108', 'RG_GEN_Rooftop_Tp114', 'RG_GEN_Rooftop_Tp120', 'RG_GEN_Rooftop_Tp126', 'RG_GEN_Rooftop_Tp132', 'RG_GEN_Rooftop_Tp138', 'RG_GEN_Rooftop_Tp144', 'RG_GEN_Rooftop_Tp150', 'RG_GEN_Rooftop_Tp156', 'RG_GEN_Rooftop_Tp162', 'RG_GEN_Rooftop_Tp168'])

    forecast_rooftop = {}
    for region in REGIONIDS:
        for i, lag in enumerate(range(6, 168+6, 6)):
            forecast_rooftop[f'{region}_GEN_Rooftop_Tp{lag}'] = df.iloc[i][region]

    return current_rooftop | forecast_rooftop 

features = features | get_rooftop_pv()

## Interconnector forecasts... TODO

not sure this is the right data either ??? seems to only be at one of these types of model scenario:'RELIABILITY_LRC', 'OUTAGE_LRC', 'LOR'. [Link to explain?](https://aemo.com.au/-/media/archive/files/electricity/consultations/2015/attachment-1-rsig.pdf) . so probably not a reliable indicator of IC_NET in future. 

In [ ]:
# filename = get_latest_file_from_folder('Short_Term_PASA_Reports')
# df = pd.read_csv(get_subtable_from_file(filename, 'STPASA,INTERCONNECTORSOLN'),
#                  # usecols=['REGIONID', 'INTERVAL_DATETIME', 'AGGREGATECAPACITYAVAILABLE', 'DEMAND50', 
#                  #          'SS_SOLAR_UIGF', 'SS_WIND_UIGF'],
#                  # parse_dates=['INTERVAL_DATETIME'],
#                  # index_col='INTERVAL_DATETIME',
#                 )
# df.RUNTYPE.unique()
# df

In [ ]:
# df[df['RUNTYPE']=='RELIABILITY_LRC'].set_index('INTERVAL_DATETIME').pivot(columns=['INTERCONNECTORID'], values=['CALCULATEDEXPORTLIMIT','CALCULATEDIMPORTLIMIT']).plot()

### alternative source... but no... werid data :(
NET_INTERCHANGE == IC_NET * -1


TODO: this seems to suck

In [ ]:
# filename = download_and_unzip_file('http://www.nemweb.com.au/REPORTS/CURRENT/SEVENDAYOUTLOOK_FULL/PUBLIC_SEVENDAYOUTLOOK_FULL_20220810231348_0000000368596516.zip')
# df = pd.read_csv(filename, 
#                  skiprows=1,
#                  skipfooter=1, engine='python', # need engine=python to use skipfooter
#                  parse_dates=['INTERVAL_DATETIME'],
#                  index_col='INTERVAL_DATETIME', 
#                  usecols=['INTERVAL_DATETIME', 'REGIONID', 'NET_INTERCHANGE']
#                 )

# # IC_NET is defined to be NETINTERCHANGE * -1. IC_NET: import is +ve generation.
# df['NET_INTERCHANGE'] = df['NET_INTERCHANGE'] * -1
# df = df.rename(columns={'NET_INTERCHANGE': 'IC_NET'})
# df = df.pivot(columns='REGIONID', values='IC_NET')

# print_item(df)
# df.plot()

## Greenness mean of last 30 days
Inputs:
- Again, use the "daily report" (also used by 30d price lags): (all 30 days available from one folder)
  - gen by fuel: This time we want a different subtable `DUNIT`, which has all the data by DUID. 
  - net interconnector flow: `DREGION` subtable
- Rooftop PV Actual (not forecast) data - in its own separate dataset (only 14 (?) days available from current folder, go to archive for the remanider)

First, rooftop PV:

### Get last 30d of rooftop data

In [ ]:

def extract_row_from_pv_file(csv_file):
    df = pd.read_csv(csv_file,
                     skiprows=1,
                     skipfooter=1,
                     engine='python',
                     usecols=['INTERVAL_DATETIME', 'REGIONID', 'POWER'],
                     parse_dates=['INTERVAL_DATETIME'],
                     index_col='REGIONID')
    row = {f'{region}_GEN_Rooftop': df.at[region, 'POWER'] for region in REGIONIDS}
    row['SETTLEMENTDATE'] = df.at['VIC1', 'INTERVAL_DATETIME']
    return row


def get_recent_weeks_of_pv_data():
    # download the 4 most recent weeks of data from the archive folder and extract into a dataframe
    rows = []
    PV_CACHE_FOLDER = DATA_FOLDER / 'download_cache/RooftopPVActual/'
    if not os.path.exists(PV_CACHE_FOLDER):
        os.makedirs(PV_CACHE_FOLDER)
    for nth_most_recent in [-4, -3, -2, -1]:

        for retries in range(5):
            response = requests.get('http://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/')
            if (len(response.text) > 2000): # predipatch table is 3100 or so
                break
        soup = BeautifulSoup(response.text, features='lxml')
        latest = soup.find_all('a')[nth_most_recent]['href']
        zip_url = f'http://nemweb.com.au{latest}'
        filename = zip_url.split('/')[-1]
        filepath = PV_CACHE_FOLDER / filename
        print(f"Downloading {zip_url}")
        # extract the .zip contents (which is more .zip files, extract and parse them too)
        response = requests.get(zip_url)
        # with open(DATA_FOLDER / 'download_cache/PUBLIC_ROOFTOP_PV_ACTUAL_SATELLITE_20220728.zip', 'rb') as f:
        #     temp = f.read()

        with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
        # with zipfile.ZipFile(io.BytesIO(temp)) as outer_zip:
            # outer_zip contains a bunch more zip files (only). Read these into memory too. 
            for inner_zip_name in outer_zip.namelist():
                with zipfile.ZipFile(outer_zip.open(inner_zip_name)) as inner_zip:
                    # inner zip files contain only one file - blah.csv. Read this into memory. 
                    with inner_zip.open(inner_zip.namelist()[0]) as csv_file:
                        rows.append(extract_row_from_pv_file(csv_file))

    df = pd.DataFrame(rows).set_index('SETTLEMENTDATE')
    return df



In [ ]:
# Get a whole day of PV actual data
# This data is midnight-to-midnight, unlike the data we're joinign to which is 4am-4am, but we can assume there's 0 PV between midnight and 4am!!!  :D
# downloads all the files (one per half-hour) for the given day 
def get_PV_actual_for_one_day(target_date):
    
    # There are both "satellite" and "measurement" files. Two measurements of same thing. Assume we want satellite. 
    match_str = f"SATELLITE_{target_date.strftime('%Y%m%d')}"  # eg 'SATELLITE_20220813'
    filenames = get_files_from_folder('ROOFTOP_PV/ACTUAL', match_str)    
    
    # one row (file) per half-hour timestamp
    rows = []
    for filename in filenames:
        df = pd.read_csv(filename, skiprows=1, parse_dates=['INTERVAL_DATETIME'])
        df = df.set_index('REGIONID')
        row = {f"{region}_GEN_Rooftop": df.at[region,'POWER'] for region in REGIONIDS}
        row['SETTLEMENTDATE'] = df.at['VIC1', 'INTERVAL_DATETIME']
        rows.append(row)
    df = pd.DataFrame(rows)
    df = df.set_index('SETTLEMENTDATE')
    return df


In [ ]:
# get last 30d of rooftop pvdata, 
# starts by either loading from disk if available or otherwise grabs week-large chunks from the nem archive
# then tops up either with most recent data
def get_last_30d_of_pv_data():
    
    if os.path.exists(DATA_FOLDER / 'greenness_rooftop_30d_cache.csv'):
        df = pd.read_csv(DATA_FOLDER / 'greenness_rooftop_30d_cache.csv',
                         parse_dates=['SETTLEMENTDATE'],
                         index_col='SETTLEMENTDATE')
    else:
        print("No Cache; running get_recent_weeks_of_pv_data()")
        df = get_recent_weeks_of_pv_data()

    # if the data in the cache is old for some reason, just refresh entirely.
    if datetime.now().date() - df.index[-1].date() > timedelta(days=10):
        print("Cache is old; refresh it with get_recent_weeks_of_pv_data()")
        df = get_recent_weeks_of_pv_data()
    
    # delete anything from today's date  # TODO really no point to grabbing in day sized chunks any more...
    df = df[df.index < pd.Timestamp.now().floor('D')]

    # now get any remaining recent days. If getting from NEM archive, should be . 
    # start with the first missing date:
    d = df.index[-1].date() + timedelta(days=1)
    days = []
    while d <= datetime.now().date():
        print("get_PV_actual_for_one_day(" + d.strftime('%Y%m%d') + ")")
        days.append(get_PV_actual_for_one_day(d))
        d = d + timedelta(days=1)

    days.append(df)

    greenness_rooftop = pd.concat(days).sort_index()
    
    #drop duplicates
    greenness_rooftop = greenness_rooftop[~greenness_rooftop.index.duplicated(keep='first')]

    greenness_rooftop.to_csv(DATA_FOLDER / 'greenness_rooftop_30d_cache.csv')
    return greenness_rooftop



### Gen-by-fuel for greenness
Then we grab all the generation-by-fuel data (and add in PV rooftop from above along the way)

In [23]:
with open('test_features.json') as f:
    feat = json.load(f)

In [25]:
[x for x in feat if 'Green' in x]

['NSW1_Greenness_Tm3d_Mean',
 'NSW1_Greenness_Tm7d_Mean',
 'NSW1_Greenness_Tm30d_Mean',
 'QLD1_Greenness_Tm3d_Mean',
 'QLD1_Greenness_Tm7d_Mean',
 'QLD1_Greenness_Tm30d_Mean',
 'SA1_Greenness_Tm3d_Mean',
 'SA1_Greenness_Tm7d_Mean',
 'SA1_Greenness_Tm30d_Mean',
 'TAS1_Greenness_Tm3d_Mean',
 'TAS1_Greenness_Tm7d_Mean',
 'TAS1_Greenness_Tm30d_Mean',
 'VIC1_Greenness_Tm3d_Mean',
 'VIC1_Greenness_Tm7d_Mean',
 'VIC1_Greenness_Tm30d_Mean']

In [ ]:
def get_gen_by_duid_for_one_day(filename, duids):
    df = pd.read_csv(get_subtable_from_file(filename, ',DUNIT,,3'),
                     usecols=['SETTLEMENTDATE', 'DUID', 'TOTALCLEARED'],
                     parse_dates=['SETTLEMENTDATE'],
                     index_col='SETTLEMENTDATE'
                    )
    df = df.rename(columns={'TOTALCLEARED': 'MW'})
    
    df = df.join(duids, on='DUID')
    
    return df
    
''' 
assumes gen_by_duid is a dataframe with columns ['SETTLEMENTDATE', 'DUID', 'MW', 'CO2E_ENERGY_SOURCE']
'''
def calculate_greenness_raw_for_one_day(gen_by_duid, greenness_rooftop):

    df = gen_by_duid.copy()
    
    # drop any loads, only want generators
    df = df[df.GENSETTYPE == 'GENERATOR']

    # no negatives (saw one once somewhere else, some solar unit in qld)
    df['MW'] = df['MW'].clip(lower=0)

    # Group by fuel type
    df = df.groupby(['SETTLEMENTDATE', 'REGIONID', 'CO2E_ENERGY_SOURCE']).agg({
        'MW': np.sum,
    }).reset_index()

    # # take average across the whole day
    # df = df.groupby(['REGIONID', 'CO2E_ENERGY_SOURCE']).agg({
    #     'TOTALCLEARED': np.mean,
    # }).reset_index()
    df = df.set_index('SETTLEMENTDATE')

    # feature names are eg VIC1_GEN_Coal
    df['name'] = df['REGIONID'] + '_GEN_' + df['CO2E_ENERGY_SOURCE']

    df = df.pivot(columns=['name'], values='MW')

    # change from 30min to 5min frequency to match PV_forecast
    df = df.resample('30min', origin='start').mean()
    df.index = df.index + pd.tseries.frequencies.to_offset('25min')  # because SETTLEMENTDATE is end-of-period

    # add in rooftop PV data
    df = df.join(greenness_rooftop)

    # we can assume 0 PV between midnight and 4am!!!  :D
    print(f"isna:{df.isna().sum().sum()}")
    if df.isna().sum().sum() >= 70:
        print(df)
    assert df.isna().sum().sum() < 70, "Some NAs in PV data is ok because they're at night, but this is too many"
    df = df.fillna(0)

    for region in REGIONIDS:
        all_gen = [f'{region}_GEN_{fuel}' for fuel in ['Coal', 'Gas', 'Hydro', 'Solar', 'Wind', 'Rooftop']]
        renewable = [f'{region}_GEN_{fuel}' for fuel in ['Hydro', 'Solar', 'Wind', 'Rooftop']]

        # add in any missing data, eg SA coal, set to zeros. 
        for feature in all_gen:
            if feature not in df.columns:
                df[feature] = 0

        df[f'{region}_Greenness_raw'] = df[renewable].sum(axis=1) / df[all_gen].sum(axis=1)            
    
    return df

Now, need interconnector flows for the last month too.

In [ ]:
# Get interconnector net for a recent trading day
def get_ics_for_one_day(filename):
    df = pd.read_csv(get_subtable_from_file(filename, ',DREGION,,3'),
                     usecols=['REGIONID', 'NETINTERCHANGE', 'SETTLEMENTDATE', ],
                     parse_dates=['SETTLEMENTDATE'],
                     index_col='SETTLEMENTDATE'
                    )
    # IC_NET is defined to be NETINTERCHANGE * -1. IC_NET: import is +ve generation.
    df['NETINTERCHANGE'] = df['NETINTERCHANGE'] * -1
    
    df = df.pivot(columns='REGIONID', values='NETINTERCHANGE')
    df.columns = [f'{region}_IC_NET' for region in df.columns]

    # change from 30min to 5min frequency to match PV_forecast
    df = df.resample('30min', origin='start').mean()
    df.index = df.index + pd.tseries.frequencies.to_offset('25min')  # because SETTLEMENTDATE is end-of-period
    return df

Finally we put it all together - gen by fuel (incl rooftop pv) & net interchange. 

In [ ]:
def calculate_real_greenness(greenness_inputs):
    greenness = pd.DataFrame(index=greenness_inputs.index)


    ############
    ### Copied from dataset generator. Consider refactoring if making changes. 

    # Now, un-raw it by adding in interconnector effects
    # calcaulte "IC_Import" which is all interconnector imports, ignoring any exports
    # This is because any exports won't change the greenness for this state
    # (this is a bit of an approximation? but close enough since IC flows don't dominate vic/nsw)
    # Then, calculate the amount of green energy flowing in, which is IC_import * greenness_raw from the incoming state.
    # this is complex for VIC & NSW, easier for other 3 states with only one feeder state. 
    # with IC_Import and IC_Green_In, we can then calculate the greenness.
    gi = greenness_inputs

    # QLD/SA/TAS are simpler
    gi['QLD1_IC_Import'] = gi['QLD1_IC_NET'].clip(lower=0)
    gi['SA1_IC_Import'] = gi['SA1_IC_NET'].clip(lower=0)
    gi['TAS1_IC_Import'] = gi['TAS1_IC_NET'].clip(lower=0)

    gi['QLD1_IC_Green_In'] = gi['QLD1_IC_Import'] * gi['NSW1_Greenness_raw']
    gi['SA1_IC_Green_In'] = gi['SA1_IC_Import'] * gi['VIC1_Greenness_raw']
    gi['TAS1_IC_Green_In'] = gi['TAS1_IC_Import'] * gi['VIC1_Greenness_raw']

    # NSW1: 
    gi['NSW1_Import_From_Qld'] = gi['QLD1_IC_NET'].clip(upper=0) * -1
    gi['NSW1_IC_Net_From_Vic'] = gi['NSW1_IC_NET'] - (-1) * gi['QLD1_IC_NET']
    gi['NSW1_Import_From_Vic'] = gi['NSW1_IC_Net_From_Vic'].clip(lower=0)
    gi['NSW1_IC_Import'] = gi['NSW1_Import_From_Qld'] + gi['NSW1_Import_From_Vic']
    gi['NSW1_IC_Green_In'] = (gi['NSW1_Import_From_Qld'] * gi['QLD1_Greenness_raw']+ 
                              gi['NSW1_Import_From_Vic'] * gi['VIC1_Greenness_raw'])

    # VIC1:
    gi['VIC1_Import_From_Tas'] = gi['TAS1_IC_NET'].clip(upper=0) * -1
    gi['VIC1_Import_From_SA']  = gi['SA1_IC_NET'].clip(upper=0) * -1
    gi['VIC1_Import_From_NSW'] = gi['NSW1_IC_Net_From_Vic'].clip(upper=0) * -1
    gi['VIC1_IC_Import'] = gi['VIC1_Import_From_Tas'] + gi['VIC1_Import_From_SA'] + gi['VIC1_Import_From_NSW']
    gi['VIC1_IC_Green_In'] = (gi['VIC1_Import_From_Tas'] * gi['TAS1_Greenness_raw'] +
                              gi['VIC1_Import_From_SA']  * gi['SA1_Greenness_raw'] +
                              gi['VIC1_Import_From_NSW'] * gi['NSW1_Greenness_raw'])


    # calculate greenness for real now we have IC_Import and IC_Green_In
    for region in REGIONIDS:
        # greenness_inputs[f'{region}_IC_Import'].plot(figsize=(16,5))

        renewable = [f'{region}_GEN_{fuel}' for fuel in ['Hydro', 'Solar', 'Wind', 'Rooftop']] + [f'{region}_IC_Green_In']
        all_gen = [f'{region}_GEN_{fuel}' for fuel in ['Coal', 'Gas', 'Hydro', 'Solar', 'Wind', 'Rooftop']] + [f'{region}_IC_Import']

        # convert to nparray with dtype=object to avoid a warning being thrown
        all_gen = np.array(all_gen, dtype=object)
        renewable = np.array(renewable, dtype=object)

        greenness[f'{region}_Greenness'] = gi[renewable].sum(axis=1) / gi[all_gen].sum(axis=1) 

    # Let's express everything as a percentage not [0,1]
    greenness = greenness * 100

    return greenness


### Calculate Greenness_Tm30d_Mean

In [ ]:
# First, get PV data:
greenness_rooftop = get_last_30d_of_pv_data()

print('Get last 30 days of daily reports:')
filenames =[get_latest_file_from_folder('Daily_Reports', nth_most_recent=i) 
            for i in range(-1, -31, -1)]

greenness_month = []  # later becomes a df
for filename in filenames:
    # filename = filenames[0]

    gen_by_duid = get_gen_by_duid_for_one_day(filename, duids)

    greenness_inputs = calculate_greenness_raw_for_one_day(gen_by_duid, greenness_rooftop)

    greenness_inputs = greenness_inputs.join(get_ics_for_one_day(filename))

    greenness_month.append(calculate_real_greenness(greenness_inputs))

greenness_month = pd.concat(greenness_month)

for region in REGIONIDS:
    now_minus_3d = greenness_month.index.max() - pd.Timedelta(3, 'D')
    features[f'{region}_Greenness_Tm3d_Mean'] = greenness_month[now_minus_3d:].mean()[f'{region}_Greenness']
    now_minus_7d = greenness_month.index.max() - pd.Timedelta(7, 'D')
    features[f'{region}_Greenness_Tm7d_Mean'] = greenness_month[now_minus_7d:].mean()[f'{region}_Greenness']
    features[f'{region}_Greenness_Tm30d_Mean'] = greenness_month.mean()[f'{region}_Greenness']

len(features)

## Greenness at current time

In [ ]:
# calcuate greenness_raw using features already available
greenness_inputs = pd.DataFrame([features])

for region in REGIONIDS:
    all_gen = [f'{region}_GEN_{fuel}' for fuel in ['Coal', 'Gas', 'Hydro', 'Solar', 'Wind', 'Rooftop']]
    renewable = [f'{region}_GEN_{fuel}' for fuel in ['Hydro', 'Solar', 'Wind', 'Rooftop']]

    greenness_inputs[f'{region}_Greenness_raw'] = greenness_inputs[renewable].sum(axis=1) / greenness_inputs[all_gen].sum(axis=1)            

    
greenness_now = calculate_real_greenness(greenness_inputs)
for col in greenness_now.columns:
    features[col] = greenness_now.iloc[0][col]
    
    
# return features

# features
len(features)

## Check

In [ ]:
all_features = [x for x in columns if 'RG_' not in x]
for region in REGIONIDS:
    all_features = all_features + [x.replace('RG_', f'{region}_') for x in columns if 'RG_' in x]
    
len(all_features), len(features), [x for x in all_features if x not in features.keys()]

# Load Models 

### Testing

In [ ]:
column_names_84 = ['VIC1_AVAILABILITY_Battery Storage', 'VIC1_AVAILABILITY_Coal', 'VIC1_AVAILABILITY_Gas', 'VIC1_AVAILABILITY_Hydro', 'VIC1_AVAILABILITY_Solar', 'VIC1_AVAILABILITY_Wind', 'VIC1_AVAILABLEGENERATION', 'VIC1_AVAILABLEGENERATION_Tp102', 'VIC1_AVAILABLEGENERATION_Tp108', 'VIC1_AVAILABLEGENERATION_Tp60', 'VIC1_AVAILABLEGENERATION_Tp66', 'VIC1_AVAILABLEGENERATION_Tp72', 'VIC1_AVAILABLEGENERATION_Tp78', 'VIC1_AVAILABLEGENERATION_Tp84', 'VIC1_AVAILABLEGENERATION_Tp90', 'VIC1_AVAILABLEGENERATION_Tp96', 'VIC1_GENERATION', 'VIC1_GEN_Battery Storage', 'VIC1_GEN_Coal', 'VIC1_GEN_Gas', 'VIC1_GEN_Hydro', 'VIC1_GEN_Rooftop', 'VIC1_GEN_Rooftop_Tp102', 'VIC1_GEN_Rooftop_Tp108', 'VIC1_GEN_Rooftop_Tp60', 'VIC1_GEN_Rooftop_Tp66', 'VIC1_GEN_Rooftop_Tp72', 'VIC1_GEN_Rooftop_Tp78', 'VIC1_GEN_Rooftop_Tp84', 'VIC1_GEN_Rooftop_Tp90', 'VIC1_GEN_Rooftop_Tp96', 'VIC1_GEN_Solar', 'VIC1_GEN_Solar_Tp102', 'VIC1_GEN_Solar_Tp108', 'VIC1_GEN_Solar_Tp60', 'VIC1_GEN_Solar_Tp66', 'VIC1_GEN_Solar_Tp72', 'VIC1_GEN_Solar_Tp78', 'VIC1_GEN_Solar_Tp84', 'VIC1_GEN_Solar_Tp90', 'VIC1_GEN_Solar_Tp96', 'VIC1_GEN_Wind', 'VIC1_GEN_Wind_Tp102', 'VIC1_GEN_Wind_Tp108', 'VIC1_GEN_Wind_Tp60', 'VIC1_GEN_Wind_Tp66', 'VIC1_GEN_Wind_Tp72', 'VIC1_GEN_Wind_Tp78', 'VIC1_GEN_Wind_Tp84', 'VIC1_GEN_Wind_Tp90', 'VIC1_GEN_Wind_Tp96', 'VIC1_Greenness', 'VIC1_Greenness_Tm30d_Mean', 'VIC1_Greenness_Tm3d_Mean', 'VIC1_Greenness_Tm7d_Mean', 'VIC1_IC_Export_Limit', 'VIC1_IC_Import_Limit', 'VIC1_IC_NET', 'VIC1_Predis_Price_Tp12', 'VIC1_Predis_Price_Tp16', 'VIC1_Predis_Price_Tp4', 'VIC1_Predis_Price_Tp8', 'VIC1_Predis_Price_max16to40h', 'VIC1_Predis_Price_min16to40h', 'VIC1_Price', 'VIC1_Price_Tm1', 'VIC1_Price_Tm10', 'VIC1_Price_Tm12', 'VIC1_Price_Tm14', 'VIC1_Price_Tm16', 'VIC1_Price_Tm168', 'VIC1_Price_Tm18', 'VIC1_Price_Tm2', 'VIC1_Price_Tm20', 'VIC1_Price_Tm22', 'VIC1_Price_Tm24', 'VIC1_Price_Tm28', 'VIC1_Price_Tm30d_15thP', 'VIC1_Price_Tm30d_85thP', 'VIC1_Price_Tm30d_Median', 'VIC1_Price_Tm32', 'VIC1_Price_Tm36', 'VIC1_Price_Tm4', 'VIC1_Price_Tm40', 'VIC1_Price_Tm44', 'VIC1_Price_Tm48', 'VIC1_Price_Tm52', 'VIC1_Price_Tm56', 'VIC1_Price_Tm6', 'VIC1_Price_Tm60', 'VIC1_Price_Tm64', 'VIC1_Price_Tm68', 'VIC1_Price_Tm72', 'VIC1_Price_Tm7d_15thP', 'VIC1_Price_Tm7d_85thP', 'VIC1_Price_Tm7d_Median', 'VIC1_Price_Tm8', 'VIC1_TOTALDEMAND', 'VIC1_TOTALDEMAND_Tp102', 'VIC1_TOTALDEMAND_Tp108', 'VIC1_TOTALDEMAND_Tp60', 'VIC1_TOTALDEMAND_Tp66', 'VIC1_TOTALDEMAND_Tp72', 'VIC1_TOTALDEMAND_Tp78', 'VIC1_TOTALDEMAND_Tp84', 'VIC1_TOTALDEMAND_Tp90', 'VIC1_TOTALDEMAND_Tp96', 'VIC1_W_day_max_temperature', 'VIC1_W_day_max_temperature_Tp72', 'VIC1_W_day_max_temperature_Tp96', 'VIC1_W_temperature', 'VIC1_is_workday', 'VIC1_is_workday_Tp72', 'VIC1_is_workday_Tp96', 'hour', 'hours_since_2010', 'is_weekend', 'month', 'quarter', 'weekday', 'year']
test_vals_84 = [0.0, 6165.0, 1330.0, 1917.0, 0.0, 0.0, 9412.0, 9492.0, 9531.0, 10030.0, 10030.0, 10020.0, 9650.0, 10020.0, 10010.0, 9825.0, 6018.0, 0.0, 6018.0, 0.666700005531311, 1.8950000666958025e-14, 0.0, 0.06875000149011612, 4.566999912261963, 11.510000228881836, 11.579999923706055, 3.1459999084472656, 0.13750000298023224, 6.711999893188477, 7.136000156402588, 1.312000036239624, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6165000200271606, 4.0320000648498535, 4.007999897003174, 4.5929999351501465, -1352.0, 428.29998779296875, -1229.0, 30.6299991607666, 52.5099983215332, 12.029999732971191, 23.389999389648438, 52.5099983215332, 17.90999984741211, 18.899999618530273, 17.559999465942383, 52.95000076293945, 43.75, 36.349998474121094, 21.540000915527344, 19.440000534057617, 18.549999237060547, 14.989999771118164, 17.68000030517578, 19.799999237060547, 26.639999389648438, 40.209999084472656, 15.539999961853027, 35.0099983215332, 22.280000686645508, 46.150001525878906, 25.06999969482422, 28.670000076293945, 21.139999389648438, 9.100000381469727, 19.100000381469727, 21.420000076293945, 29.600000381469727, 29.700000762939453, 30.18000030517578, 23.989999771118164, 18.299999237060547, 20.149999618530273, 18.450000762939453, 37.91999816894531, 26.059999465942383, 39.70000076293945, 4790.0, 5791.0, 6561.0, 8516.0, 8855.0, 8650.0, 6440.0, 7872.0, 7579.0, 6782.0, 22.780000686645508, 26.889999389648438, 20.43000030517578, 17.959999084472656, 1.0, 1.0, 1.0, 0.0, 8640.0, 0.0, 2.0, 1.0, 0.0, 0.0]
test_features_84 = {col:val for col,val in list(zip(column_names_84, test_vals_84))}

column_names = ['VIC1_AVAILABILITY_Battery Storage', 'VIC1_AVAILABILITY_Coal', 'VIC1_AVAILABILITY_Gas', 'VIC1_AVAILABILITY_Hydro', 'VIC1_AVAILABILITY_Solar', 'VIC1_AVAILABILITY_Wind', 'VIC1_AVAILABLEGENERATION', 'VIC1_AVAILABLEGENERATION_Tp102', 'VIC1_AVAILABLEGENERATION_Tp108', 'VIC1_AVAILABLEGENERATION_Tp114', 'VIC1_AVAILABLEGENERATION_Tp12', 'VIC1_AVAILABLEGENERATION_Tp120', 'VIC1_AVAILABLEGENERATION_Tp126', 'VIC1_AVAILABLEGENERATION_Tp132', 'VIC1_AVAILABLEGENERATION_Tp138', 'VIC1_AVAILABLEGENERATION_Tp144', 'VIC1_AVAILABLEGENERATION_Tp150', 'VIC1_AVAILABLEGENERATION_Tp156', 'VIC1_AVAILABLEGENERATION_Tp162', 'VIC1_AVAILABLEGENERATION_Tp168', 'VIC1_AVAILABLEGENERATION_Tp18', 'VIC1_AVAILABLEGENERATION_Tp24', 'VIC1_AVAILABLEGENERATION_Tp30', 'VIC1_AVAILABLEGENERATION_Tp36', 'VIC1_AVAILABLEGENERATION_Tp42', 'VIC1_AVAILABLEGENERATION_Tp48', 'VIC1_AVAILABLEGENERATION_Tp54', 'VIC1_AVAILABLEGENERATION_Tp6', 'VIC1_AVAILABLEGENERATION_Tp60', 'VIC1_AVAILABLEGENERATION_Tp66', 'VIC1_AVAILABLEGENERATION_Tp72', 'VIC1_AVAILABLEGENERATION_Tp78', 'VIC1_AVAILABLEGENERATION_Tp84', 'VIC1_AVAILABLEGENERATION_Tp90', 'VIC1_AVAILABLEGENERATION_Tp96', 'VIC1_GENERATION', 'VIC1_GEN_Battery Storage', 'VIC1_GEN_Coal', 'VIC1_GEN_Gas', 'VIC1_GEN_Hydro', 'VIC1_GEN_Rooftop', 'VIC1_GEN_Rooftop_Tp102', 'VIC1_GEN_Rooftop_Tp108', 'VIC1_GEN_Rooftop_Tp114', 'VIC1_GEN_Rooftop_Tp12', 'VIC1_GEN_Rooftop_Tp120', 'VIC1_GEN_Rooftop_Tp126', 'VIC1_GEN_Rooftop_Tp132', 'VIC1_GEN_Rooftop_Tp138', 'VIC1_GEN_Rooftop_Tp144', 'VIC1_GEN_Rooftop_Tp150', 'VIC1_GEN_Rooftop_Tp156', 'VIC1_GEN_Rooftop_Tp162', 'VIC1_GEN_Rooftop_Tp168', 'VIC1_GEN_Rooftop_Tp18', 'VIC1_GEN_Rooftop_Tp24', 'VIC1_GEN_Rooftop_Tp30', 'VIC1_GEN_Rooftop_Tp36', 'VIC1_GEN_Rooftop_Tp42', 'VIC1_GEN_Rooftop_Tp48', 'VIC1_GEN_Rooftop_Tp54', 'VIC1_GEN_Rooftop_Tp6', 'VIC1_GEN_Rooftop_Tp60', 'VIC1_GEN_Rooftop_Tp66', 'VIC1_GEN_Rooftop_Tp72', 'VIC1_GEN_Rooftop_Tp78', 'VIC1_GEN_Rooftop_Tp84', 'VIC1_GEN_Rooftop_Tp90', 'VIC1_GEN_Rooftop_Tp96', 'VIC1_GEN_Solar', 'VIC1_GEN_Solar_Tp102', 'VIC1_GEN_Solar_Tp108', 'VIC1_GEN_Solar_Tp114', 'VIC1_GEN_Solar_Tp12', 'VIC1_GEN_Solar_Tp120', 'VIC1_GEN_Solar_Tp126', 'VIC1_GEN_Solar_Tp132', 'VIC1_GEN_Solar_Tp138', 'VIC1_GEN_Solar_Tp144', 'VIC1_GEN_Solar_Tp150', 'VIC1_GEN_Solar_Tp156', 'VIC1_GEN_Solar_Tp162', 'VIC1_GEN_Solar_Tp168', 'VIC1_GEN_Solar_Tp18', 'VIC1_GEN_Solar_Tp24', 'VIC1_GEN_Solar_Tp30', 'VIC1_GEN_Solar_Tp36', 'VIC1_GEN_Solar_Tp42', 'VIC1_GEN_Solar_Tp48', 'VIC1_GEN_Solar_Tp54', 'VIC1_GEN_Solar_Tp6', 'VIC1_GEN_Solar_Tp60', 'VIC1_GEN_Solar_Tp66', 'VIC1_GEN_Solar_Tp72', 'VIC1_GEN_Solar_Tp78', 'VIC1_GEN_Solar_Tp84', 'VIC1_GEN_Solar_Tp90', 'VIC1_GEN_Solar_Tp96', 'VIC1_GEN_Wind', 'VIC1_GEN_Wind_Tp102', 'VIC1_GEN_Wind_Tp108', 'VIC1_GEN_Wind_Tp114', 'VIC1_GEN_Wind_Tp12', 'VIC1_GEN_Wind_Tp120', 'VIC1_GEN_Wind_Tp126', 'VIC1_GEN_Wind_Tp132', 'VIC1_GEN_Wind_Tp138', 'VIC1_GEN_Wind_Tp144', 'VIC1_GEN_Wind_Tp150', 'VIC1_GEN_Wind_Tp156', 'VIC1_GEN_Wind_Tp162', 'VIC1_GEN_Wind_Tp168', 'VIC1_GEN_Wind_Tp18', 'VIC1_GEN_Wind_Tp24', 'VIC1_GEN_Wind_Tp30', 'VIC1_GEN_Wind_Tp36', 'VIC1_GEN_Wind_Tp42', 'VIC1_GEN_Wind_Tp48', 'VIC1_GEN_Wind_Tp54', 'VIC1_GEN_Wind_Tp6', 'VIC1_GEN_Wind_Tp60', 'VIC1_GEN_Wind_Tp66', 'VIC1_GEN_Wind_Tp72', 'VIC1_GEN_Wind_Tp78', 'VIC1_GEN_Wind_Tp84', 'VIC1_GEN_Wind_Tp90', 'VIC1_GEN_Wind_Tp96', 'VIC1_Greenness', 'VIC1_Greenness_Tm30d_Mean', 'VIC1_Greenness_Tm3d_Mean', 'VIC1_Greenness_Tm7d_Mean', 'VIC1_IC_Export_Limit', 'VIC1_IC_Import_Limit', 'VIC1_IC_NET', 'VIC1_Predis_Price_Tp12', 'VIC1_Predis_Price_Tp16', 'VIC1_Predis_Price_Tp4', 'VIC1_Predis_Price_Tp8', 'VIC1_Predis_Price_max16to40h', 'VIC1_Predis_Price_min16to40h', 'VIC1_Price', 'VIC1_Price_Tm1', 'VIC1_Price_Tm10', 'VIC1_Price_Tm12', 'VIC1_Price_Tm14', 'VIC1_Price_Tm16', 'VIC1_Price_Tm168', 'VIC1_Price_Tm18', 'VIC1_Price_Tm2', 'VIC1_Price_Tm20', 'VIC1_Price_Tm22', 'VIC1_Price_Tm24', 'VIC1_Price_Tm28', 'VIC1_Price_Tm30d_15thP', 'VIC1_Price_Tm30d_85thP', 'VIC1_Price_Tm30d_Median', 'VIC1_Price_Tm32', 'VIC1_Price_Tm36', 'VIC1_Price_Tm4', 'VIC1_Price_Tm40', 'VIC1_Price_Tm44', 'VIC1_Price_Tm48', 'VIC1_Price_Tm52', 'VIC1_Price_Tm56', 'VIC1_Price_Tm6', 'VIC1_Price_Tm60', 'VIC1_Price_Tm64', 'VIC1_Price_Tm68', 'VIC1_Price_Tm72', 'VIC1_Price_Tm7d_15thP', 'VIC1_Price_Tm7d_85thP', 'VIC1_Price_Tm7d_Median', 'VIC1_Price_Tm8', 'VIC1_Price_Tp84', 'VIC1_TOTALDEMAND', 'VIC1_TOTALDEMAND_Tp102', 'VIC1_TOTALDEMAND_Tp108', 'VIC1_TOTALDEMAND_Tp114', 'VIC1_TOTALDEMAND_Tp12', 'VIC1_TOTALDEMAND_Tp120', 'VIC1_TOTALDEMAND_Tp126', 'VIC1_TOTALDEMAND_Tp132', 'VIC1_TOTALDEMAND_Tp138', 'VIC1_TOTALDEMAND_Tp144', 'VIC1_TOTALDEMAND_Tp150', 'VIC1_TOTALDEMAND_Tp156', 'VIC1_TOTALDEMAND_Tp162', 'VIC1_TOTALDEMAND_Tp168', 'VIC1_TOTALDEMAND_Tp18', 'VIC1_TOTALDEMAND_Tp24', 'VIC1_TOTALDEMAND_Tp30', 'VIC1_TOTALDEMAND_Tp36', 'VIC1_TOTALDEMAND_Tp42', 'VIC1_TOTALDEMAND_Tp48', 'VIC1_TOTALDEMAND_Tp54', 'VIC1_TOTALDEMAND_Tp6', 'VIC1_TOTALDEMAND_Tp60', 'VIC1_TOTALDEMAND_Tp66', 'VIC1_TOTALDEMAND_Tp72', 'VIC1_TOTALDEMAND_Tp78', 'VIC1_TOTALDEMAND_Tp84', 'VIC1_TOTALDEMAND_Tp90', 'VIC1_TOTALDEMAND_Tp96', 'VIC1_W_day_max_temperature', 'VIC1_W_day_max_temperature_Tp120', 'VIC1_W_day_max_temperature_Tp144', 'VIC1_W_day_max_temperature_Tp168', 'VIC1_W_day_max_temperature_Tp24', 'VIC1_W_day_max_temperature_Tp48', 'VIC1_W_day_max_temperature_Tp72', 'VIC1_W_day_max_temperature_Tp96', 'VIC1_W_temperature', 'VIC1_is_workday', 'VIC1_is_workday_Tp120', 'VIC1_is_workday_Tp144', 'VIC1_is_workday_Tp168', 'VIC1_is_workday_Tp24', 'VIC1_is_workday_Tp48', 'VIC1_is_workday_Tp72', 'VIC1_is_workday_Tp96', 'day', 'hour', 'hours_since_2010', 'is_augment_row', 'is_validation_set', 'is_weekend', 'month', 'quarter', 'validation_set', 'weekday', 'year']
test_vals = [0.0, 6165.0, 1330.0, 1917.0, 0.0, 0.0, 9412.0, 9492.0, 9531.0, 9563.0, 9477.0, 9465.0, 9116.0, 8975.0, 9228.0, 9269.0, 9680.0, 9672.0, 9692.0, 9694.0, 9448.0, 9462.0, 9477.0, 10010.0, 10070.0, 10080.0, 9612.0, 9473.0, 10030.0, 10030.0, 10020.0, 9650.0, 10020.0, 10010.0, 9825.0, 6018.0, 0.0, 6018.0, 0.666700005531311, 1.8950000666958025e-14, 0.0, 0.06875000149011612, 4.566999912261963, 5.7210001945495605, 11.199999809265137, 2.4059998989105225, 0.2062000036239624, 8.664999961853027, 9.326000213623047, 3.382999897003174, 0.22920000553131104, 11.680000305175781, 12.010000228881836, 3.509999990463257, 11.229999542236328, 3.430999994277954, 0.3021000027656555, 10.539999961853027, 10.770000457763672, 2.808000087738037, 0.3021000027656555, 0.3021000027656555, 11.510000228881836, 11.579999923706055, 3.1459999084472656, 0.13750000298023224, 6.711999893188477, 7.136000156402588, 1.312000036239624, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6165000200271606, 4.0320000648498535, 4.007999897003174, 4.5929999351501465, -1352.0, 428.29998779296875, -1229.0, 30.6299991607666, 52.5099983215332, 12.029999732971191, 23.389999389648438, 52.5099983215332, 17.90999984741211, 18.899999618530273, 17.559999465942383, 52.95000076293945, 43.75, 36.349998474121094, 21.540000915527344, 19.440000534057617, 18.549999237060547, 14.989999771118164, 17.68000030517578, 19.799999237060547, 26.639999389648438, 40.209999084472656, 15.539999961853027, 35.0099983215332, 22.280000686645508, 46.150001525878906, 25.06999969482422, 28.670000076293945, 21.139999389648438, 9.100000381469727, 19.100000381469727, 21.420000076293945, 29.600000381469727, 29.700000762939453, 30.18000030517578, 23.989999771118164, 18.299999237060547, 20.149999618530273, 18.450000762939453, 37.91999816894531, 26.059999465942383, 39.70000076293945, 21.809999465942383, 4790.0, 5791.0, 6561.0, 6461.0, 6883.0, 5713.0, 4801.0, 5533.0, 5648.0, 5403.0, 4725.0, 5917.0, 7087.0, 7012.0, 7161.0, 6429.0, 5664.0, 8121.0, 8868.0, 8310.0, 5971.0, 5470.0, 8516.0, 8855.0, 8650.0, 6440.0, 7872.0, 7579.0, 6782.0, 22.780000686645508, 24.6299991607666, 28.34000015258789, 31.389999389648438, 32.08000183105469, 31.920000076293945, 26.889999389648438, 20.43000030517578, 17.959999084472656, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 8640.0, 0.0, 1.0, 0.0, 2.0, 1.0, 2.0, 0.0, 0.0]
test_features = {col:val for col,val in list(zip(column_names, test_vals))}
# Add a 2nd region to the test dataset.
other_region = {f"TAS1_{col[5:]}": test_features[col] for col in column_names if 'VIC1_' in col}
test_features = test_features | other_region 

def get_test_features():
    return test_features

In [ ]:
# test: predict NN with dummy values
learner = load_fastai_nn_model('VIC1_Price_Tp84')

column_names_84 = ['VIC1_AVAILABILITY_Battery Storage', 'VIC1_AVAILABILITY_Coal', 'VIC1_AVAILABILITY_Gas', 'VIC1_AVAILABILITY_Hydro', 'VIC1_AVAILABILITY_Solar', 'VIC1_AVAILABILITY_Wind', 'VIC1_AVAILABLEGENERATION', 'VIC1_AVAILABLEGENERATION_Tp102', 'VIC1_AVAILABLEGENERATION_Tp108', 'VIC1_AVAILABLEGENERATION_Tp60', 'VIC1_AVAILABLEGENERATION_Tp66', 'VIC1_AVAILABLEGENERATION_Tp72', 'VIC1_AVAILABLEGENERATION_Tp78', 'VIC1_AVAILABLEGENERATION_Tp84', 'VIC1_AVAILABLEGENERATION_Tp90', 'VIC1_AVAILABLEGENERATION_Tp96', 'VIC1_GENERATION', 'VIC1_GEN_Battery Storage', 'VIC1_GEN_Coal', 'VIC1_GEN_Gas', 'VIC1_GEN_Hydro', 'VIC1_GEN_Rooftop', 'VIC1_GEN_Rooftop_Tp102', 'VIC1_GEN_Rooftop_Tp108', 'VIC1_GEN_Rooftop_Tp60', 'VIC1_GEN_Rooftop_Tp66', 'VIC1_GEN_Rooftop_Tp72', 'VIC1_GEN_Rooftop_Tp78', 'VIC1_GEN_Rooftop_Tp84', 'VIC1_GEN_Rooftop_Tp90', 'VIC1_GEN_Rooftop_Tp96', 'VIC1_GEN_Solar', 'VIC1_GEN_Solar_Tp102', 'VIC1_GEN_Solar_Tp108', 'VIC1_GEN_Solar_Tp60', 'VIC1_GEN_Solar_Tp66', 'VIC1_GEN_Solar_Tp72', 'VIC1_GEN_Solar_Tp78', 'VIC1_GEN_Solar_Tp84', 'VIC1_GEN_Solar_Tp90', 'VIC1_GEN_Solar_Tp96', 'VIC1_GEN_Wind', 'VIC1_GEN_Wind_Tp102', 'VIC1_GEN_Wind_Tp108', 'VIC1_GEN_Wind_Tp60', 'VIC1_GEN_Wind_Tp66', 'VIC1_GEN_Wind_Tp72', 'VIC1_GEN_Wind_Tp78', 'VIC1_GEN_Wind_Tp84', 'VIC1_GEN_Wind_Tp90', 'VIC1_GEN_Wind_Tp96', 'VIC1_Greenness', 'VIC1_Greenness_Tm30d_Mean', 'VIC1_Greenness_Tm3d_Mean', 'VIC1_Greenness_Tm7d_Mean', 'VIC1_IC_Export_Limit', 'VIC1_IC_Import_Limit', 'VIC1_IC_NET', 'VIC1_Predis_Price_Tp12', 'VIC1_Predis_Price_Tp16', 'VIC1_Predis_Price_Tp4', 'VIC1_Predis_Price_Tp8', 'VIC1_Predis_Price_max16to40h', 'VIC1_Predis_Price_min16to40h', 'VIC1_Price', 'VIC1_Price_Tm1', 'VIC1_Price_Tm10', 'VIC1_Price_Tm12', 'VIC1_Price_Tm14', 'VIC1_Price_Tm16', 'VIC1_Price_Tm168', 'VIC1_Price_Tm18', 'VIC1_Price_Tm2', 'VIC1_Price_Tm20', 'VIC1_Price_Tm22', 'VIC1_Price_Tm24', 'VIC1_Price_Tm28', 'VIC1_Price_Tm30d_15thP', 'VIC1_Price_Tm30d_85thP', 'VIC1_Price_Tm30d_Median', 'VIC1_Price_Tm32', 'VIC1_Price_Tm36', 'VIC1_Price_Tm4', 'VIC1_Price_Tm40', 'VIC1_Price_Tm44', 'VIC1_Price_Tm48', 'VIC1_Price_Tm52', 'VIC1_Price_Tm56', 'VIC1_Price_Tm6', 'VIC1_Price_Tm60', 'VIC1_Price_Tm64', 'VIC1_Price_Tm68', 'VIC1_Price_Tm72', 'VIC1_Price_Tm7d_15thP', 'VIC1_Price_Tm7d_85thP', 'VIC1_Price_Tm7d_Median', 'VIC1_Price_Tm8', 'VIC1_TOTALDEMAND', 'VIC1_TOTALDEMAND_Tp102', 'VIC1_TOTALDEMAND_Tp108', 'VIC1_TOTALDEMAND_Tp60', 'VIC1_TOTALDEMAND_Tp66', 'VIC1_TOTALDEMAND_Tp72', 'VIC1_TOTALDEMAND_Tp78', 'VIC1_TOTALDEMAND_Tp84', 'VIC1_TOTALDEMAND_Tp90', 'VIC1_TOTALDEMAND_Tp96', 'VIC1_W_day_max_temperature', 'VIC1_W_day_max_temperature_Tp72', 'VIC1_W_day_max_temperature_Tp96', 'VIC1_W_temperature', 'VIC1_is_workday', 'VIC1_is_workday_Tp72', 'VIC1_is_workday_Tp96', 'hour', 'hours_since_2010', 'is_weekend', 'month', 'quarter', 'weekday', 'year']
test_vals_84 = [0.0, 6165.0, 1330.0, 1917.0, 0.0, 0.0, 9412.0, 9492.0, 9531.0, 10030.0, 10030.0, 10020.0, 9650.0, 10020.0, 10010.0, 9825.0, 6018.0, 0.0, 6018.0, 0.666700005531311, 1.8950000666958025e-14, 0.0, 0.06875000149011612, 4.566999912261963, 11.510000228881836, 11.579999923706055, 3.1459999084472656, 0.13750000298023224, 6.711999893188477, 7.136000156402588, 1.312000036239624, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6165000200271606, 4.0320000648498535, 4.007999897003174, 4.5929999351501465, -1352.0, 428.29998779296875, -1229.0, 30.6299991607666, 52.5099983215332, 12.029999732971191, 23.389999389648438, 52.5099983215332, 17.90999984741211, 18.899999618530273, 17.559999465942383, 52.95000076293945, 43.75, 36.349998474121094, 21.540000915527344, 19.440000534057617, 18.549999237060547, 14.989999771118164, 17.68000030517578, 19.799999237060547, 26.639999389648438, 40.209999084472656, 15.539999961853027, 35.0099983215332, 22.280000686645508, 46.150001525878906, 25.06999969482422, 28.670000076293945, 21.139999389648438, 9.100000381469727, 19.100000381469727, 21.420000076293945, 29.600000381469727, 29.700000762939453, 30.18000030517578, 23.989999771118164, 18.299999237060547, 20.149999618530273, 18.450000762939453, 37.91999816894531, 26.059999465942383, 39.70000076293945, 4790.0, 5791.0, 6561.0, 8516.0, 8855.0, 8650.0, 6440.0, 7872.0, 7579.0, 6782.0, 22.780000686645508, 26.889999389648438, 20.43000030517578, 17.959999084472656, 1.0, 1.0, 1.0, 0.0, 8640.0, 0.0, 2.0, 1.0, 0.0, 0.0]

X = pd.Series(test_vals_84, index=column_names_84)
learner.predict(X)[2].item()
# 32.810882568359375

In [ ]:
# test: predict XGB with dummy values
xgb = load_xgb_model('VIC1_Price_Tp84')

column_names_84 = ['VIC1_AVAILABILITY_Battery Storage', 'VIC1_AVAILABILITY_Coal', 'VIC1_AVAILABILITY_Gas', 'VIC1_AVAILABILITY_Hydro', 'VIC1_AVAILABILITY_Solar', 'VIC1_AVAILABILITY_Wind', 'VIC1_AVAILABLEGENERATION', 'VIC1_AVAILABLEGENERATION_Tp102', 'VIC1_AVAILABLEGENERATION_Tp108', 'VIC1_AVAILABLEGENERATION_Tp60', 'VIC1_AVAILABLEGENERATION_Tp66', 'VIC1_AVAILABLEGENERATION_Tp72', 'VIC1_AVAILABLEGENERATION_Tp78', 'VIC1_AVAILABLEGENERATION_Tp84', 'VIC1_AVAILABLEGENERATION_Tp90', 'VIC1_AVAILABLEGENERATION_Tp96', 'VIC1_GENERATION', 'VIC1_GEN_Battery Storage', 'VIC1_GEN_Coal', 'VIC1_GEN_Gas', 'VIC1_GEN_Hydro', 'VIC1_GEN_Rooftop', 'VIC1_GEN_Rooftop_Tp102', 'VIC1_GEN_Rooftop_Tp108', 'VIC1_GEN_Rooftop_Tp60', 'VIC1_GEN_Rooftop_Tp66', 'VIC1_GEN_Rooftop_Tp72', 'VIC1_GEN_Rooftop_Tp78', 'VIC1_GEN_Rooftop_Tp84', 'VIC1_GEN_Rooftop_Tp90', 'VIC1_GEN_Rooftop_Tp96', 'VIC1_GEN_Solar', 'VIC1_GEN_Solar_Tp102', 'VIC1_GEN_Solar_Tp108', 'VIC1_GEN_Solar_Tp60', 'VIC1_GEN_Solar_Tp66', 'VIC1_GEN_Solar_Tp72', 'VIC1_GEN_Solar_Tp78', 'VIC1_GEN_Solar_Tp84', 'VIC1_GEN_Solar_Tp90', 'VIC1_GEN_Solar_Tp96', 'VIC1_GEN_Wind', 'VIC1_GEN_Wind_Tp102', 'VIC1_GEN_Wind_Tp108', 'VIC1_GEN_Wind_Tp60', 'VIC1_GEN_Wind_Tp66', 'VIC1_GEN_Wind_Tp72', 'VIC1_GEN_Wind_Tp78', 'VIC1_GEN_Wind_Tp84', 'VIC1_GEN_Wind_Tp90', 'VIC1_GEN_Wind_Tp96', 'VIC1_Greenness', 'VIC1_Greenness_Tm30d_Mean', 'VIC1_Greenness_Tm3d_Mean', 'VIC1_Greenness_Tm7d_Mean', 'VIC1_IC_Export_Limit', 'VIC1_IC_Import_Limit', 'VIC1_IC_NET', 'VIC1_Predis_Price_Tp12', 'VIC1_Predis_Price_Tp16', 'VIC1_Predis_Price_Tp4', 'VIC1_Predis_Price_Tp8', 'VIC1_Predis_Price_max16to40h', 'VIC1_Predis_Price_min16to40h', 'VIC1_Price', 'VIC1_Price_Tm1', 'VIC1_Price_Tm10', 'VIC1_Price_Tm12', 'VIC1_Price_Tm14', 'VIC1_Price_Tm16', 'VIC1_Price_Tm168', 'VIC1_Price_Tm18', 'VIC1_Price_Tm2', 'VIC1_Price_Tm20', 'VIC1_Price_Tm22', 'VIC1_Price_Tm24', 'VIC1_Price_Tm28', 'VIC1_Price_Tm30d_15thP', 'VIC1_Price_Tm30d_85thP', 'VIC1_Price_Tm30d_Median', 'VIC1_Price_Tm32', 'VIC1_Price_Tm36', 'VIC1_Price_Tm4', 'VIC1_Price_Tm40', 'VIC1_Price_Tm44', 'VIC1_Price_Tm48', 'VIC1_Price_Tm52', 'VIC1_Price_Tm56', 'VIC1_Price_Tm6', 'VIC1_Price_Tm60', 'VIC1_Price_Tm64', 'VIC1_Price_Tm68', 'VIC1_Price_Tm72', 'VIC1_Price_Tm7d_15thP', 'VIC1_Price_Tm7d_85thP', 'VIC1_Price_Tm7d_Median', 'VIC1_Price_Tm8', 'VIC1_TOTALDEMAND', 'VIC1_TOTALDEMAND_Tp102', 'VIC1_TOTALDEMAND_Tp108', 'VIC1_TOTALDEMAND_Tp60', 'VIC1_TOTALDEMAND_Tp66', 'VIC1_TOTALDEMAND_Tp72', 'VIC1_TOTALDEMAND_Tp78', 'VIC1_TOTALDEMAND_Tp84', 'VIC1_TOTALDEMAND_Tp90', 'VIC1_TOTALDEMAND_Tp96', 'VIC1_W_day_max_temperature', 'VIC1_W_day_max_temperature_Tp72', 'VIC1_W_day_max_temperature_Tp96', 'VIC1_W_temperature', 'VIC1_is_workday', 'VIC1_is_workday_Tp72', 'VIC1_is_workday_Tp96', 'hour', 'hours_since_2010', 'is_weekend', 'month', 'quarter', 'weekday', 'year']
test_vals_84 = [0.0, 6165.0, 1330.0, 1917.0, 0.0, 0.0, 9412.0, 9492.0, 9531.0, 10030.0, 10030.0, 10020.0, 9650.0, 10020.0, 10010.0, 9825.0, 6018.0, 0.0, 6018.0, 0.666700005531311, 1.8950000666958025e-14, 0.0, 0.06875000149011612, 4.566999912261963, 11.510000228881836, 11.579999923706055, 3.1459999084472656, 0.13750000298023224, 6.711999893188477, 7.136000156402588, 1.312000036239624, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6165000200271606, 4.0320000648498535, 4.007999897003174, 4.5929999351501465, -1352.0, 428.29998779296875, -1229.0, 30.6299991607666, 52.5099983215332, 12.029999732971191, 23.389999389648438, 52.5099983215332, 17.90999984741211, 18.899999618530273, 17.559999465942383, 52.95000076293945, 43.75, 36.349998474121094, 21.540000915527344, 19.440000534057617, 18.549999237060547, 14.989999771118164, 17.68000030517578, 19.799999237060547, 26.639999389648438, 40.209999084472656, 15.539999961853027, 35.0099983215332, 22.280000686645508, 46.150001525878906, 25.06999969482422, 28.670000076293945, 21.139999389648438, 9.100000381469727, 19.100000381469727, 21.420000076293945, 29.600000381469727, 29.700000762939453, 30.18000030517578, 23.989999771118164, 18.299999237060547, 20.149999618530273, 18.450000762939453, 37.91999816894531, 26.059999465942383, 39.70000076293945, 4790.0, 5791.0, 6561.0, 8516.0, 8855.0, 8650.0, 6440.0, 7872.0, 7579.0, 6782.0, 22.780000686645508, 26.889999389648438, 20.43000030517578, 17.959999084472656, 1.0, 1.0, 1.0, 0.0, 8640.0, 0.0, 2.0, 1.0, 0.0, 0.0]

X = pd.DataFrame([test_vals_84], columns=column_names_84)
# X = xgboost.DMatrix(np.array(vals).reshape(1,286), label=column_names)
xgb.predict(X)[0]
# 55.13684

# Prediction

In [ ]:
def load_fastai_nn_model(model_id):
    return fastai.load_learner(MODELS_FOLDER / f"{model_id}_nn.pkl")

def load_xgb_model(model_id):
    xgb = xgboost.XGBRegressor()
    xgb.load_model(MODELS_FOLDER / f"{model_id}_xgb.txt")
    xgb.predictor = 'cpu_predictor'  # would be gpu_predictor otherwise, don't know if gpu is available on lambda
    return xgb

def predict_fastai_nn(model_id, data):
    model = load_fastai_nn_model(model_id)
    features = pd.Series(data)
    return model.predict(features)[2].item()

def predict_xgb(model_id, data):
    model = load_xgb_model(model_id)
    features = pd.DataFrame([data])
    return model.predict(features)[0]




In [ ]:
def make_forecast(fc_name, all_features):
    predictions = {}
    for fc_time in FORECAST_TIMES:
        model_id = f"{fc_name}_Tp{fc_time}"
        
        print(model_id + " ", end="")
        
        # get feature names for this particular model_id - by loading the fastai model and asking it!
        temp_learner = load_fastai_nn_model(model_id)
        data_for_this_model = {key: features[key] for key in temp_learner.dls.cont_names}
        
        # convenience: gather all used feature names, can be used to check
        # global all_used_feature_names
        # all_used_feature_names = all_used_feature_names | set(temp_learner.dls.cont_names)
        
        continue
        predictions[f"{model_id}_xgb"] = predict_xgb(model_id, data_for_this_model)
        predictions[f"{model_id}_nn"] = predict_fastai_nn(model_id, data_for_this_model)
        predictions[f"{model_id}_pred"] = (predictions[f"{model_id}_xgb"] * (1-ENSEMBLE_RATIO) + 
                                           predictions[f"{model_id}_nn"] * ENSEMBLE_RATIO)
    return predictions

# log_features_and_predictions_to_db() takes the features pulled from aemo and the 
# results of the predictions and saves them to dynamodb for reference later. 
# features = {'VIC1_Price': 100, ...}
# a forecast is a dict of predictions, forecasts is a dict of forecasts
def log_features_and_predictions_to_db(base_time, features, forecasts):
    # join each forecast together into a single dict
    # predictions = {'VIC1_Price_Tp2_pred': 1000, ... 'VIC1_Greenness_Tp168_pred': 1000, ... 'NSW1_Price_Tp84_pred': 100, ...}
    predictions = {}
    for preds in forecasts.values():
        predictions = predictions | preds
        
    # join input features and predictions
    all_features = dict(sorted(list(features.items()) + list(predictions.items())))
    
    print(f"TODO: write to database {len(all_features)} features, which includes {len(features)} features.")
    
    # write all_features to database. 
    # key is current time we're forecasting from,
    # value is all_features

# interpolate_forecasts() puts predictions into a dataframe, adding an absolute time index on the way
def interpolate_forecasts(forecasts, base_time): 
    # turn each forecast dict into a list, taking only the emsembled prediction ('..._pred') not 
    # the raw xgb/nn for each time.
    preds_only = {}
    for fc_name, fc in forecasts.items():
        preds_only[fc_name] = [val for key, val in fc.items() if '_pred' in key]
    df = pd.DataFrame(preds_only, index=FORECAST_TIMES)
    
    # change to datetime index
    df.index = [base_time + pd.Timedelta(hours, 'H') for hours in df.index]
    
    # interpolate
    df = df.resample('1H').interpolate(method='spline', order=3)

    # format for output
    output = {
        'BaseTime': base_time.isoformat(),
        'DateTime': pd.to_datetime(df.index).map(lambda x: x.isoformat()).to_list(),
        'Forecasts': {col: df[col].values.round(2).tolist() for col in df.columns},
    }
    return output

# deploy_forecasts() exports all the forecasts just made to S3
# TODO: where to upload this stuff? where will it be cached??
def deploy_forecasts(forecasts, filename):
    
    with open(filename, 'w') as f:
        json.dump(forecasts, f)
    
    

### Main()

File format for forecasts: 'latest_forecasts.json':

```
{ "BaseTime": "2022-09-26T15:00:00",
  "DateTime": ["2022-09-26T17:00:00", ..., "2022-10-03T15:00:00"],
  "Forecasts": {
    "VIC1_Price": [10.0, ... 11.0],
    ...
  }}
```

In [ ]:
# TODO... make sure this is the right time...
base_time = pd.Timestamp.round(pd.Timestamp.now(), 'H')

# collect all the current data 
features = get_test_features()

# make forecasts
forecasts = {}
# for fc in FORECASTS_TO_MAKE:
for fc in ['VIC1_Price', 'VIC1_Greenness']: #, 'TAS1_Price', 'TAS1_Greenness']:
    forecasts[fc] = make_forecast(fc, features)

interpolated = interpolate_forecasts(forecasts, base_time)

deploy_forecasts(interpolated, 'latest_forecasts.json')


with open('latest_forecasts.json') as f:
    tmp = json.loads(f.read())
print(json.dumps(tmp))

# save raw data to db
log_features_and_predictions_to_db(base_time, features, forecasts)


In [ ]:
pd.DataFrame(interpolated['Forecasts'], index=interpolated['DateTime']).plot()

# Graveyard

In [ ]:
model_files = [x for x in os.listdir() if '.json' in x]
models = {x[:-5]: load_xgb_model(x) for x in model_files[:20]}
models

In [ ]:
ppjson(output)

In [ ]:
print(json.dumps(json.loads(df.to_json(orient='columns', date_format='iso', date_unit='s')), indent=2))

In [ ]:
# convert each forecast to just a single ordered list of values, all combined into one dict with the forecast name as key
{ column: df[column].values.tolist() for column in df.columns}


In [ ]:
{key: list(preds.values()) for key, preds in forecasts.items()}

In [ ]:
column_names = list(models.values())[0].get_booster().feature_names
len(column_names)

In [ ]:
column_names = models['VIC1_Greenness_Tp13'].get_booster().feature_names
vals = [1.0/len(x) for x in models['VIC1_Greenness_Tp13'].get_booster().feature_names]

X = pd.DataFrame([vals], columns=column_names)
# X = xgboost.DMatrix(np.array(vals).reshape(1,286), label=column_names)
models['VIC1_Greenness_Tp13'].predict(X)

In [ ]:
for model in models.values():
    print(model.predict(X))

### Get list of y_names we want to predict

In [ ]:
forecasts = [f"{region}_{

In [ ]:
predictions_to_make = []
for price_or_greenness in ['Price', 'Greenness']:
    for region in REGIONIDS:
        for model_type in ['fastai_nn', 'xgboost']:
            for forecast in FORECAST_TIMES:
                predictions_to_make.append(f'{region}_{price_or_greenness}_Tp{forecast}